<a href="https://colab.research.google.com/github/antsvelez/birdagentai/blob/main/BirdAgentAI_Code_4_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import { createContext, useContext, useState, useEffect } from 'react';
import { useAuth } from '../firebase/auth';
import axios from 'axios';
import { ToastContext } from '../contexts/ToastContext';
import { ErrorBoundary } from '../components/ErrorBoundary';
import { useAudio } from '../hooks/useAudio';

// API Error Types
export const API_ERROR_TYPES = {
  UNAUTHORIZED: 'UNAUTHORIZED',
  FORBIDDEN: 'FORBIDDEN',
  SERVER_ERROR: 'SERVER_ERROR',
  CONNECTION_ERROR: 'CONNECTION_ERROR',
  VALIDATION_ERROR: 'VALIDATION_ERROR',
  UNKNOWN: 'UNKNOWN'
};

// API Error Class
class APIError extends Error {
  constructor(message, type, details = null) {
    super(message);
    this.type = type;
    this.details = details;
    this.name = 'APIError';
  }
}

// Create API context
const ApiContext = createContext();

// API provider component
export const ApiProvider = ({ children }) => {
  const { getIdToken } = useAuth();
  const { showToast } = useContext(ToastContext);
  const { playSound } = useAudio('error');
  const [apiClient, setApiClient] = useState(null);
  const [error, setError] = useState(null);

  // Initialize API client with authentication
  useEffect(() => {
    const client = axios.create({
      baseURL: '/api',
      headers: {
        'Content-Type': 'application/json',
        'Accept': 'application/json'
      },
      timeout: 30000 // 30 seconds timeout
    });

    // Add request interceptor to add auth token
    client.interceptors.request.use(
      async (config) => {
        try {
          const token = await getIdToken();
          if (token) {
            config.headers.Authorization = `Bearer ${token}`;
          }
          return config;
        } catch (error) {
          console.error('Error getting auth token:', error);
          setError(new APIError(
            'Failed to get authentication token',
            API_ERROR_TYPES.UNAUTHORIZED,
            error
          ));
          playSound();
          return config;
        }
      },
      (error) => {
        setError(new APIError(
          'Failed to set up request',
          API_ERROR_TYPES.CONNECTION_ERROR,
          error
        ));
        playSound();
        return Promise.reject(error);
      }
    );

    // Add response interceptor for error handling
    client.interceptors.response.use(
      (response) => {
        // Validate response data
        if (response.data && response.data.error) {
          throw new APIError(
            response.data.error,
            API_ERROR_TYPES.VALIDATION_ERROR,
            response.data
          );
        }
        return response;
      },
      (error) => {
        let apiError;
        if (error.response) {
          switch (error.response.status) {
            case 401:
              apiError = new APIError(
                'Unauthorized access. Please log in again.',
                API_ERROR_TYPES.UNAUTHORIZED,
                error.response.data
              );
              break;
            case 403:
              apiError = new APIError(
                'Access forbidden. You do not have permission for this action.',
                API_ERROR_TYPES.FORBIDDEN,
                error.response.data
              );
              break;
            case 400:
              apiError = new APIError(
                'Bad request. Please check your input.',
                API_ERROR_TYPES.VALIDATION_ERROR,
                error.response.data
              );
              break;
            case 500:
              apiError = new APIError(
                'Server error. Please try again later.',
                API_ERROR_TYPES.SERVER_ERROR,
                error.response.data
              );
              break;
            default:
              apiError = new APIError(
                `API Error: ${error.response.data?.message || 'Unknown error'}`,
                API_ERROR_TYPES.UNKNOWN,
                error.response.data
              );
          }
        } else if (error.request) {
          apiError = new APIError(
            'No response received from server. Please check your connection.',
            API_ERROR_TYPES.CONNECTION_ERROR,
            error.request
          );
        } else {
          apiError = new APIError(
            `Error setting up request: ${error.message}`,
            API_ERROR_TYPES.UNKNOWN,
            error
          );
        }

        setError(apiError);
        playSound();
        showToast({
          message: apiError.message,
          type: 'error'
        });
        return Promise.reject(apiError);
      }
    );

    setApiClient(client);

    // Cleanup on unmount
    return () => {
      client.interceptors.request.clear();
      client.interceptors.response.clear();
    };
  }, [getIdToken, showToast, playSound]);

  // API services
  const services = {
    // Patients API
    patients: {
      getAll: async (params = {}) => {
        try {
          const response = await apiClient.get('/patients', { params });
          return response.data;
        } catch (error) {
          throw error;
        }
      },
      getById: async (id) => {
        try {
          const response = await apiClient.get(`/patients/${id}`);
          return response.data;
        } catch (error) {
          throw error;
        }
      },
      create: async (data) => {
        try {
          const response = await apiClient.post('/patients', data);
          return response.data;
        } catch (error) {
          throw error;
        }
      },
      update: async (id, data) => {
        try {
          const response = await apiClient.put(`/patients/${id}`, data);
          return response.data;
        } catch (error) {
          throw error;
        }
      },
      delete: async (id) => {
        try {
          await apiClient.delete(`/patients/${id}`);
          return true;
        } catch (error) {
          throw error;
        }
      }
    },

    // Appointments API
    appointments: {
      getAll: async (params = {}) => {
        try {
          const response = await apiClient.get('/appointments', { params });
          return response.data;
        } catch (error) {
          throw error;
        }
      },
      getByPatient: async (patientId) => {
        try {
          const response = await apiClient.get(`/appointments?patientId=${patientId}`);
          return response.data;
        } catch (error) {
          throw error;
        }
      },
      getById: async (id) => {
        try {
          const response = await apiClient.get(`/appointments/${id}`);
          return response.data;
        } catch (error) {
          throw error;
        }
      },
      create: async (data) => {
        try {
          const response = await apiClient.post('/appointments', data);
          return response.data;
        } catch (error) {
          throw error;
        }
      },
      update: async (id, data) => {
        try {
          const response = await apiClient.put(`/appointments/${id}`, data);
          return response.data;
        } catch (error) {
          throw error;
        }
      },
      delete: async (id) => {
        try {
          await apiClient.delete(`/appointments/${id}`);
          return true;
        } catch (error) {
          throw error;
        }
      }
    },

    // Leads API
    leads: {
      getAll: async (params = {}) => {
        try {
          const response = await apiClient.get('/leads', { params });
          return response.data;
        } catch (error) {
          throw error;
        }
      },
      getByPatient: async (patientId) => {
        try {
          const response = await apiClient.get(`/leads?patientId=${patientId}`);
          return response.data;
        } catch (error) {
          throw error;
        }
      },
      getById: async (id) => {
        try {
          const response = await apiClient.get(`/leads/${id}`);
          return response.data;
        } catch (error) {
          throw error;
        }
      },
      create: async (data) => {
        try {
          const response = await apiClient.post('/leads', data);
          return response.data;
        } catch (error) {
          throw error;
        }
      },
      update: async (id, data) => {
        try {
          const response = await apiClient.put(`/leads/${id}`, data);
          return response.data;
        } catch (error) {
          throw error;
        }
      },
      delete: async (id) => {
        try {
          await apiClient.delete(`/leads/${id}`);
          return true;
        } catch (error) {
          throw error;
        }
      }
    },

    // Calls API
    calls: {
      getAll: async (params = {}) => {
        try {
          const response = await apiClient.get('/calls', { params });
          return response.data;
        } catch (error) {
          throw error;
        }
      },
      getByPatient: async (patientId) => {
        try {
          const response = await apiClient.get(`/calls?patientId=${patientId}`);
          return response.data;
        } catch (error) {
          throw error;
        }
      },
      getById: async (id) => {
        try {
          const response = await apiClient.get(`/calls/${id}`);
          return response.data;
        } catch (error) {
          throw error;
        }
      },
      create: async (data) => {
        try {
          const response = await apiClient.post('/calls', data);
          return response.data;
        } catch (error) {
          throw error;
        }
      },
      update: async (id, data) => {
        try {
          const response = await apiClient.put(`/calls/${id}`, data);
          return response.data;
        } catch (error) {
          throw error;
        }
      },
      complete: async (id, transcript) => {
        try {
          const response = await apiClient.post(`/calls/${id}/complete`, { transcript });
          return response.data;
        } catch (error) {
          throw error;
        }
      },
      delete: async (id) => {
        try {
          await apiClient.delete(`/calls/${id}`);
          return true;
        } catch (error) {
          throw error;
        }
      }
    },

    // User API
    user: {
      getProfile: async () => {
        try {
          const response = await apiClient.get('/auth/profile');
          return response.data;
        } catch (error) {
          throw error;
        }
      },
      updateProfile: async (data) => {
        try {
          const response = await apiClient.put('/auth/profile', data);
          return response.data;
        } catch (error) {
          throw error;
        }
      },
      getAllUsers: async () => {
        try {
          const response = await apiClient.get('/auth/users');
          return response.data;
        } catch (error) {
          throw error;
        }
      }
    }
  };

  return (
    <ErrorBoundary>
      <ApiContext.Provider value={{ apiClient, ...services, error }}>
        {children}
      </ApiContext.Provider>
    </ErrorBoundary>
  );
};

// Custom hook to use API context
export const useApi = () => {
  const context = useContext(ApiContext);
  if (!context) {
    throw new Error('useApi must be used within an ApiProvider');
  }
  return context;
};

// Error handling utility
export const handleApiError = (error) => {
  if (error instanceof APIError) {
    switch (error.type) {
      case API_ERROR_TYPES.UNAUTHORIZED:
        // Handle unauthorized access
        break;
      case API_ERROR_TYPES.FORBIDDEN:
        // Handle forbidden access
        break;
      case API_ERROR_TYPES.SERVER_ERROR:
        // Handle server errors
        break;
      case API_ERROR_TYPES.CONNECTION_ERROR:
        // Handle connection issues
        break;
      case API_ERROR_TYPES.VALIDATION_ERROR:
        // Handle validation errors
        break;
      default:
        // Handle unknown errors
        break;
    }
  }
  throw error;
};


In [ ]:
import React from 'react';
import { BrowserRouter as Router, Routes, Route, Navigate } from 'react-router-dom';
import { AuthProvider } from './firebase/auth';
import { ApiProvider } from './services/api';
import PrivateRoute from './components/common/PrivateRoute';
import Layout from './components/layout/Layout';
import Login from './components/auth/Login';
import Register from './components/auth/Register';
import Dashboard from './components/dashboard/Dashboard';
import Patients from './components/patients/Patients';
import PatientDetail from './components/patients/PatientDetail';
import Appointments from './components/appointments/Appointments';
import Leads from './components/leads/Leads';
import Calls from './components/calls/Calls';
import Profile from './components/profile/Profile';

function App() {
  return (
    <AuthProvider>
      <ApiProvider>
        <Router>
          <Routes>
            <Route path="/login" element={<Login />} />
            <Route path="/register" element={<Register />} />
            <Route path="/" element={
              <PrivateRoute>
                <Layout />
              </PrivateRoute>
            }>
              <Route index element={<Dashboard />} />
              <Route path="patients" element={<Patients />} />
              <Route path="patients/:id" element={<PatientDetail />} />
              <Route path="appointments" element={<Appointments />} />
              <Route path="leads" element={<Leads />} />
              <Route path="calls" element={<Calls />} />
              <Route path="profile" element={<Profile />} />
              <Route path="*" element={<Navigate to="/" replace />} />
            </Route>
          </Routes>
        </Router>
      </ApiProvider>
    </AuthProvider>
  );
}

export default App;

In [ ]:
import React, { useState, useEffect, useCallback, useRef } from 'react';
import { appointmentService, patientService } from '../../firebase/firestore';
import { useApi } from '../../services/api';
import { format, isValid, parseISO } from 'date-fns';
import { useToast } from '../../components/ui/ToastContext';

const Appointments = () => {
  const [appointments, setAppointments] = useState([]);
  const [patients, setPatients] = useState([]);
  const [loading, setLoading] = useState(true);
  const [error, setError] = useState(null);
  const [showAddModal, setShowAddModal] = useState(false);
  const [newAppointment, setNewAppointment] = useState({
    patientId: '',
    appointmentType: 'checkup',
    appointmentDate: '',
    appointmentTime: '',
    status: 'scheduled',
    notes: ''
  });
  const [formErrors, setFormErrors] = useState({});
  const toast = useToast();
  const formRef = useRef(null);
  const api = useApi();

  const validateForm = useCallback(() => {
    const errors = {};
    if (!newAppointment.patientId) {
      errors.patientId = 'Please select a patient';
    }
    if (!newAppointment.appointmentDate) {
      errors.appointmentDate = 'Please select a date';
    } else if (!isValid(parseISO(newAppointment.appointmentDate))) {
      errors.appointmentDate = 'Please enter a valid date';
    }
    if (!newAppointment.appointmentTime) {
      errors.appointmentTime = 'Please select a time';
    }
    return errors;
  }, [newAppointment]);

  const fetchData = useCallback(async () => {
    try {
      setLoading(true);
      setError(null);

      const [appointmentsData, patientsData] = await Promise.all([
        appointmentService.getAllAppointments(),
        patientService.getAllPatients()
      ]);

      const sortedAppointments = appointmentsData.sort((a, b) =>
        new Date(b.appointmentDate) - new Date(a.appointmentDate)
      );

      setAppointments(sortedAppointments);
      setPatients(patientsData);
    } catch (error) {
      console.error('Error fetching data:', error);
      setError('Failed to load appointments. Please try again.');
      toast.error('Failed to load appointments');
    } finally {
      setLoading(false);
    }
  }, [toast]);

  useEffect(() => {
    fetchData();
    return () => {
      // Cleanup any subscriptions if needed
    };
  }, [fetchData]);

  const handleInputChange = useCallback((e) => {
    const { name, value } = e.target;
    setNewAppointment(prev => ({
      ...prev,
      [name]: value
    }));

    // Clear error for changed field
    setFormErrors(prev => ({
      ...prev,
      [name]: ''
    }));
  }, []);

  const handleSubmit = useCallback(async (e) => {
    e.preventDefault();

    const errors = validateForm();
    if (Object.keys(errors).length > 0) {
      setFormErrors(errors);
      return;
    }

    try {
      setLoading(true);
      setError(null);

      const dateTime = new Date(`${newAppointment.appointmentDate}T${newAppointment.appointmentTime}`);
      if (dateTime < new Date()) {
        toast.error('Cannot create appointment in the past');
        return;
      }

      const appointmentData = {
        patientId: newAppointment.patientId,
        appointmentType: newAppointment.appointmentType,
        appointmentDate: dateTime,
        status: newAppointment.status,
        notes: newAppointment.notes
      };

      const appointmentId = await appointmentService.addAppointment(appointmentData);
      const patient = patients.find(p => p.id === newAppointment.patientId);

      setAppointments(prev => [
        {
          id: appointmentId,
          ...appointmentData,
          patientName: patient ? `${patient.firstName} ${patient.lastName}` : 'Unknown Patient'
        },
        ...prev
      ]);

      toast.success('Appointment created successfully');
      setNewAppointment({
        patientId: '',
        appointmentType: 'checkup',
        appointmentDate: '',
        appointmentTime: '',
        status: 'scheduled',
        notes: ''
      });
      setFormErrors({});
      setShowAddModal(false);
    } catch (error) {
      console.error('Error adding appointment:', error);
      setError('Failed to add appointment. Please try again.');
      toast.error('Failed to create appointment');
    } finally {
      setLoading(false);
    }
  }, [newAppointment, patients, validateForm, toast]);

  const getPatientName = useCallback((patientId) => {
    const patient = patients.find(p => p.id === patientId);
    return patient ? `${patient.firstName} ${patient.lastName}` : 'Unknown Patient';
  }, [patients]);

  const renderAppointmentList = () => (
    <div className="mt-6 bg-white shadow overflow-hidden sm:rounded-md">
      <ul className="divide-y divide-gray-200">
        {appointments.length > 0 ? (
          appointments.map((appointment) => (
            <li key={appointment.id}>
              <div className="px-4 py-4 sm:px-6">
                <div className="flex items-center justify-between">
                  <div className="text-sm font-medium text-blue-600 truncate">
                    {appointment.appointmentType}
                  </div>
                  <div className="ml-2 flex-shrink-0 flex">
                    <span className={`px-2 inline-flex text-xs leading-5 font-semibold rounded-full ${
                      appointment.status === 'scheduled' ? 'bg-green-100 text-green-800' :
                      appointment.status === 'cancelled' ? 'bg-red-100 text-red-800' :
                      'bg-yellow-100 text-yellow-800'
                    }`}>
                      {appointment.status}
                    </span>
                  </div>
                </div>
                <div className="mt-2 sm:flex sm:justify-between">
                  <div className="sm:flex">
                    <div className="flex items-center text-sm text-gray-500">
                      <svg className="flex-shrink-0 mr-1.5 h-5 w-5 text-gray-400" fill="none" viewBox="0 0 24 24" stroke="currentColor">
                        <path strokeLinecap="round" strokeLinejoin="round" strokeWidth="2" d="M16 7a4 4 0 11-8 0 4 4 0 018 0zM12 14a7 7 0 00-7 7h14a7 7 0 00-7-7z" />
                      </svg>
                      <span>{getPatientName(appointment.patientId)}</span>
                    </div>
                  </div>
                  <div className="mt-2 flex items-center text-sm text-gray-500 sm:mt-0">
                    <svg className="flex-shrink-0 mr-1.5 h-5 w-5 text-gray-400" fill="none" viewBox="0 0 24 24" stroke="currentColor">
                      <path strokeLinecap="round" strokeLinejoin="round" strokeWidth="2" d="M8 7V3m8 4V3m-9 8h10M5 21h14a2 2 0 002-2V7a2 2 0 00-2-2H5a2 2 0 00-2 2v12a2 2 0 002 2z" />
                    </svg>
                    <span>
                      {format(new Date(appointment.appointmentDate), 'MMM d, yyyy')} at {format(new Date(appointment.appointmentDate), 'h:mm a')}
                    </span>
                  </div>
                </div>
                {appointment.notes && (
                  <div className="mt-2">
                    <p className="text-sm text-gray-500">{appointment.notes}</p>
                  </div>
                )}
              </div>
            </li>
          ))
        ) : (
          <li className="px-4 py-4 sm:px-6 text-sm text-gray-500">
            No appointments found.
          </li>
        )}
      </ul>
    </div>
  );

  if (loading && appointments.length === 0) {
    return (
      <div className="flex items-center justify-center h-64">
        <div className="animate-spin rounded-full h-12 w-12 border-t-2 border-b-2 border-blue-500"></div>
      </div>
    );
  }

  return (
    <div>
      <div className="flex justify-between items-center">
        <h1 className="text-2xl font-semibold text-gray-900">Appointments</h1>
        <button
          onClick={() => setShowAddModal(true)}
          className="inline-flex items-center px-4 py-2 border border-transparent text-sm font-medium rounded-md shadow-sm text-white bg-blue-600 hover:bg-blue-700 focus:outline-none focus:ring-2 focus:ring-offset-2 focus:ring-blue-500"
        >
          <svg className="-ml-1 mr-2 h-5 w-5" fill="none" viewBox="0 0 24 24" stroke="currentColor">
            <path strokeLinecap="round" strokeLinejoin="round" strokeWidth="2" d="M12 6v6m0 0v6m0-6h6m-6 0H6" />
          </svg>
          Add Appointment
        </button>
      </div>

      {error && (
        <div className="mt-4 bg-red-100 border border-red-400 text-red-700 px-4 py-3 rounded relative" role="alert">
          <span className="block sm:inline">{error}</span>
        </div>
      )}

      {renderAppointmentList()}

      {showAddModal && (
        <div className="fixed z-10 inset-0 overflow-y-auto" role="dialog" aria-modal="true" aria-labelledby="modal-title">
          <div className="flex items-end justify-center min-h-screen pt-4 px-4 pb-20 text-center sm:block sm:p-0">
            <div className="fixed inset-0 bg-gray-500 bg-opacity-75 transition-opacity" aria-hidden="true" onClick={() => setShowAddModal(false)}></div>

            <div className="inline-block align-bottom bg-white rounded-lg text-left overflow-hidden shadow-xl transform transition-all sm:my-8 sm:align-middle sm:max-w-lg sm:w-full">
              <form onSubmit={handleSubmit} ref={formRef}>
                <div className="bg-white px-4 pt-5 pb-4 sm:p-6 sm:pb-4">
                  <div className="sm:flex sm:items-start">
                    <div className="mt-3 text-center sm:mt-0 sm:ml-4 sm:text-left w-full">
                      <h3 className="text-lg leading-6 font-medium text-gray-900" id="modal-title">
                        Add New Appointment
                      </h3>
                      <div className="mt-4 grid grid-cols-1 gap-y-6 gap-x-4 sm:grid-cols-6">
                        <div className="sm:col-span-6">
                          <label htmlFor="patientId" className="block text-sm font-medium text-gray-700">
                            Patient
                          </label>
                          <div className="mt-1">
                            <select
                              id="patientId"
                              name="patientId"
                              value={newAppointment.patientId}
                              onChange={handleInputChange}
                              className="shadow-sm focus:ring-blue-500 focus:border-blue-500 block w-full sm:text-sm border-gray-300 rounded-md"
                              required
                            >
                              <option value="">Select a patient</option>
                              {patients.map(patient => (
                                <option key={patient.id} value={patient.id}>
                                  {`${patient.firstName} ${patient.lastName}`}
                                </option>
                              ))}
                            </select>
                            {formErrors.patientId && (
                              <p className="mt-1 text-sm text-red-600" id="patientId-error">
                                {formErrors.patientId}
                              </p>
                            )}
                          </div>
                        </div>

                        <div className="sm:col-span-6">
                          <label htmlFor="appointmentType" className="block text-sm font-medium text-gray-700">
                            Appointment Type
                          </label>
                          <div className="mt-1">
                            <select
                              id="appointmentType"
                              name="appointmentType"
                              value={newAppointment.appointmentType}
                              onChange={handleInputChange}
                              className="shadow-sm focus:ring-blue-500 focus:border-blue-500 block w-full sm:text-sm border-gray-300 rounded-md"
                              required
                            >
                              <option value="checkup">Checkup</option>
                              <option value="cleaning">Cleaning</option>
                              <option value="consultation">Consultation</option>
                              <option value="emergency">Emergency</option>
                            </select>
                          </div>
                        </div>

                        <div className="sm:col-span-6">
                          <label htmlFor="appointmentDate" className="block text-sm font-medium text-gray-700">
                            Appointment Date
                          </label>
                          <div className="mt-1">
                            <input
                              type="date"
                              id="appointmentDate"
                              name="appointmentDate"
                              value={newAppointment.appointmentDate}
                              onChange={handleInputChange}
                              className="shadow-sm focus:ring-blue-500 focus:border-blue-500 block w-full sm:text-sm border-gray-300 rounded-md"
                              required
                            />
                            {formErrors.appointmentDate && (
                              <p className="mt-1 text-sm text-red-600" id="appointmentDate-error">
                                {formErrors.appointmentDate}
                              </p>
                            )}
                          </div>
                        </div>

                        <div className="sm:col-span-6">
                          <label htmlFor="appointmentTime" className="block text-sm font-medium text-gray-700">
                            Appointment Time
                          </label>
                          <div className="mt-1">
                            <input
                              type="time"
                              id="appointmentTime"
                              name="appointmentTime"
                              value={newAppointment.appointmentTime}
                              onChange={handleInputChange}
                              className="shadow-sm focus:ring-blue-500 focus:border-blue-500 block w-full sm:text-sm border-gray-300 rounded-md"
                              required
                            />
                            {formErrors.appointmentTime && (
                              <p className="mt-1 text-sm text-red-600" id="appointmentTime-error">
                                {formErrors.appointmentTime}
                              </p>
                            )}
                          </div>
                        </div>

                        <div className="sm:col-span-6">
                          <label htmlFor="status" className="block text-sm font-medium text-gray-700">
                            Status
                          </label>
                          <div className="mt-1">
                            <select
                              id="status"
                              name="status"
                              value={newAppointment.status}
                              onChange={handleInputChange}
                              className="shadow-sm focus:ring-blue-500 focus:border-blue-500 block w-full sm:text-sm border-gray-300 rounded-md"
                              required
                            >
                              <option value="scheduled">Scheduled</option>
                              <option value="completed">Completed</option>
                              <option value="cancelled">Cancelled</option>
                            </select>
                          </div>
                        </div>

                        <div className="sm:col-span-6">
                          <label htmlFor="notes" className="block text-sm font-medium text-gray-700">
                            Notes
                          </label>
                          <div className="mt-1">
                            <textarea
                              id="notes"
                              name="notes"
                              rows={3}
                              value={newAppointment.notes}
                              onChange={handleInputChange}
                              className="shadow-sm focus:ring-blue-500 focus:border-blue-500 block w-full sm:text-sm border-gray-300 rounded-md"
                            />
                          </div>
                        </div>
                      </div>
                    </div>
                  </div>
                </div>
                <div className="bg-gray-50 px-4 py-3 sm:px-6 sm:flex sm:flex-row-reverse">
                  <button
                    type="submit"
                    disabled={loading}
                    className="inline-flex justify-center py-2 px-4 border border-transparent shadow-sm text-sm font-medium rounded-md text-white bg-blue-600 hover:bg-blue-700 focus:outline-none focus:ring-2 focus:ring-offset-2 focus:ring-blue-500"
                  >
                    {loading ? 'Creating...' : 'Create Appointment'}
                  </button>
                  <button
                    type="button"
                    onClick={() => setShowAddModal(false)}
                    className="mt-3 w-full inline-flex justify-center rounded-md border border-gray-300 shadow-sm px-4 py-2 bg-white text-base font-medium text-gray-700 hover:bg-gray-50 focus:outline-none focus:ring-2 focus:ring-offset-2 focus:ring-blue-500 sm:mt-0 sm:ml-3 sm:w-auto sm:text-sm"
                  >
                    Cancel
                  </button>
                </div>
              </form>
            </div>
          </div>
        </div>
      )}
    </div>
  );
};

export default Appointments;

In [ ]:
import { createContext, useContext, useState, useEffect } from 'react';
import {
  createUserWithEmailAndPassword,
  signInWithEmailAndPassword,
  signOut,
  onAuthStateChanged,
  sendPasswordResetEmail,
  updateProfile
} from 'firebase/auth';
import { doc, setDoc, getDoc, serverTimestamp } from 'firebase/firestore';
import { auth, db } from './firebase';

// Create auth context
const AuthContext = createContext();

// Auth provider component
export const AuthProvider = ({ children }) => {
  const [currentUser, setCurrentUser] = useState(null);
  const [userData, setUserData] = useState(null);
  const [loading, setLoading] = useState(true);
  const [authError, setAuthError] = useState(null);

  // Register a new user
  const register = async (email, password, firstName, lastName, role = 'staff') => {
    try {
      setAuthError(null);
      // Create user in Firebase Auth
      const userCredential = await createUserWithEmailAndPassword(auth, email, password);
      const user = userCredential.user;

      // Update display name
      await updateProfile(user, {
        displayName: `${firstName} ${lastName}`
      });

      // Create user document in Firestore
      await setDoc(doc(db, 'users', user.uid), {
        firstName,
        lastName,
        email,
        role,
        createdAt: serverTimestamp()
      });

      return user;
    } catch (error) {
      console.error('Registration error:', error);
      setAuthError(error.message);
      throw error;
    }
  };

  // Login user
  const login = async (email, password) => {
    try {
      setAuthError(null);
      const userCredential = await signInWithEmailAndPassword(auth, email, password);
      return userCredential.user;
    } catch (error) {
      console.error('Login error:', error);
      setAuthError(error.message);
      throw error;
    }
  };

  // Logout user
  const logout = async () => {
    try {
      setAuthError(null);
      await signOut(auth);
      setUserData(null);
    } catch (error) {
      console.error('Logout error:', error);
      setAuthError(error.message);
      throw error;
    }
  };

  // Reset password
  const resetPassword = async (email) => {
    try {
      setAuthError(null);
      await sendPasswordResetEmail(auth, email);
    } catch (error) {
      console.error('Password reset error:', error);
      setAuthError(error.message);
      throw error;
    }
  };

  // Update user profile
  const updateUserProfile = async (data) => {
    try {
      if (!currentUser) throw new Error('No authenticated user');

      // Update Firestore document
      await setDoc(doc(db, 'users', currentUser.uid), {
        ...data,
        updatedAt: serverTimestamp()
      }, { merge: true });

      // Update local state
      setUserData(prevData => ({
        ...prevData,
        ...data
      }));

      return true;
    } catch (error) {
      console.error('Profile update error:', error);
      throw error;
    }
  };

  // Get ID token for API calls
  const getIdToken = async () => {
    if (!currentUser) return null;
    return await currentUser.getIdToken();
  };

  // Fetch user data from Firestore
  const fetchUserData = async (user) => {
    if (!user) return null;

    try {
      const userDoc = await getDoc(doc(db, 'users', user.uid));
      if (userDoc.exists()) {
        return userDoc.data();
      }
      return null;
    } catch (error) {
      console.error('Error fetching user data:', error);
      return null;
    }
  };

  // Listen for auth state changes
  useEffect(() => {
    const unsubscribe = onAuthStateChanged(auth, async (user) => {
      setCurrentUser(user);

      if (user) {
        const userData = await fetchUserData(user);
        setUserData(userData);
      } else {
        setUserData(null);
      }

      setLoading(false);
    });

    return unsubscribe;
  }, []);

  const value = {
    currentUser,
    userData,
    loading,
    authError,
    register,
    login,
    logout,
    resetPassword,
    updateUserProfile,
    getIdToken
  };

  return (
    <AuthContext.Provider value={value}>
      {!loading && children}
    </AuthContext.Provider>
  );
};

// Custom hook to use auth context
export const useAuth = () => {
  return useContext(AuthContext);
};


In [ ]:
import React, { useState, useEffect, useCallback, useRef, useMemo } from 'react';
import { callService, patientService } from '../../firebase/firestore';
import { useApi } from '../../services/api';
import { format, isValid, parseISO } from 'date-fns';
import { useToast } from '../../components/ui/ToastContext';
import { useErrorBoundary } from '../../components/ui/ErrorBoundary';
import { useAudio } from '../../hooks/useAudio';
import { CALL_TYPES, CALL_PURPOSES } from '../../constants/callTypes';

const CALL_STATUS = {
  SCHEDULED: 'scheduled',
  IN_PROGRESS: 'in_progress',
  COMPLETED: 'completed',
  CANCELLED: 'cancelled'
};

const Calls = () => {
  const [calls, setCalls] = useState([]);
  const [patients, setPatients] = useState([]);
  const [loading, setLoading] = useState(true);
  const [error, setError] = useState(null);
  const [showAddModal, setShowAddModal] = useState(false);
  const [activeCall, setActiveCall] = useState(null);
  const [formErrors, setFormErrors] = useState({});
  const [typingAudio, setTypingAudio] = useState(null);
  const [newCall, setNewCall] = useState({
    patientId: '',
    callType: CALL_TYPES.OUTBOUND,
    callPurpose: CALL_PURPOSES.APPOINTMENT_REMINDER,
    scheduledTime: '',
    scheduledTimeHour: '09',
    scheduledTimeMinute: '00',
    callStatus: CALL_STATUS.SCHEDULED,
    notes: ''
  });
  const toast = useToast();
  const api = useApi();
  const formRef = useRef(null);
  const errorBoundary = useErrorBoundary();

  // Initialize typing sound effect with proper cleanup
  useEffect(() => {
    const audio = useAudio('/assets/audio/typing-sound.mp3');
    setTypingAudio(audio);

    return () => {
      if (audio) {
        audio.pause();
        audio.src = '';
      }
    };
  }, []);

  // Validate form inputs
  const validateForm = useCallback(() => {
    const errors = {};
    if (!newCall.patientId) {
      errors.patientId = 'Please select a patient';
    }
    if (!newCall.scheduledTime) {
      errors.scheduledTime = 'Please select a date';
    }
    if (!newCall.scheduledTimeHour || !newCall.scheduledTimeMinute) {
      errors.time = 'Please select a time';
    }
    return errors;
  }, [newCall]);

  // Fetch calls and patients with error handling
  const fetchData = useCallback(async () => {
    try {
      setLoading(true);
      setError(null);

      const [callsData, patientsData] = await Promise.all([
        callService.getAllCalls(),
        patientService.getAllPatients()
      ]);

      const sortedCalls = callsData.sort((a, b) =>
        new Date(b.scheduledTime) - new Date(a.scheduledTime)
      );

      setCalls(sortedCalls);
      setPatients(patientsData);
    } catch (error) {
      console.error('Error fetching data:', error);
      setError('Failed to load calls. Please try again.');
      toast.error('Failed to load calls');
      errorBoundary.captureError(error);
    } finally {
      setLoading(false);
    }
  }, [toast, errorBoundary]);

  useEffect(() => {
    fetchData();
    return () => {
      // Cleanup any subscriptions if needed
    };
  }, [fetchData]);

  // Handle form input changes with validation
  const handleInputChange = useCallback((e) => {
    const { name, value } = e.target;
    setNewCall(prev => ({
      ...prev,
      [name]: value
    }));

    // Clear error for changed field
    setFormErrors(prev => ({
      ...prev,
      [name]: ''
    }));
  }, []);

  // Handle form submission with validation
  const handleSubmit = useCallback(async (e) => {
    e.preventDefault();

    const errors = validateForm();
    if (Object.keys(errors).length > 0) {
      setFormErrors(errors);
      return;
    }

    try {
      setLoading(true);
      setError(null);

      const dateStr = newCall.scheduledTime;
      const timeStr = `${newCall.scheduledTimeHour}:${newCall.scheduledTimeMinute}`;
      const dateTime = new Date(`${dateStr}T${timeStr}:00`);

      if (dateTime < new Date()) {
        toast.error('Cannot schedule call in the past');
        return;
      }

      const callData = {
        patientId: newCall.patientId,
        callType: newCall.callType,
        callPurpose: newCall.callPurpose,
        scheduledTime: dateTime,
        callStatus: CALL_STATUS.SCHEDULED,
        notes: newCall.notes
      };

      const callId = await callService.addCall(callData);
      const patient = patients.find(p => p.id === newCall.patientId);

      setCalls(prev => [
        {
          id: callId,
          ...callData,
          patientName: patient ? `${patient.firstName} ${patient.lastName}` : 'Unknown Patient'
        },
        ...prev
      ]);

      toast.success('Call scheduled successfully');
      setNewCall({
        patientId: '',
        callType: CALL_TYPES.OUTBOUND,
        callPurpose: CALL_PURPOSES.APPOINTMENT_REMINDER,
        scheduledTime: '',
        scheduledTimeHour: '09',
        scheduledTimeMinute: '00',
        callStatus: CALL_STATUS.SCHEDULED,
        notes: ''
      });
      setFormErrors({});
      setShowAddModal(false);
    } catch (error) {
      console.error('Error adding call:', error);
      setError('Failed to schedule call. Please try again.');
      toast.error('Failed to schedule call');
      errorBoundary.captureError(error);
    } finally {
      setLoading(false);
    }
  }, [newCall, patients, validateForm, toast, errorBoundary]);

  // Get patient name by ID
  const getPatientName = useCallback((patientId) => {
    const patient = patients.find(p => p.id === patientId);
    return patient ? `${patient.firstName} ${patient.lastName}` : 'Unknown Patient';
  }, [patients]);

  // Handle starting a call with proper error handling
  const handleStartCall = useCallback(async (call) => {
    if (loading || !call || call.callStatus !== CALL_STATUS.SCHEDULED) return;

    try {
      setLoading(true);
      setError(null);

      await callService.updateCall(call.id, {
        ...call,
        callStatus: CALL_STATUS.IN_PROGRESS,
        startTime: new Date()
      });

      const updatedCalls = calls.map(c =>
        c.id === call.id
          ? { ...c, callStatus: CALL_STATUS.IN_PROGRESS, startTime: new Date() }
          : c
      );

      setCalls(updatedCalls);
      setActiveCall({
        ...call,
        callStatus: CALL_STATUS.IN_PROGRESS,
        startTime: new Date()
      });

      if (typingAudio) {
        typingAudio.currentTime = 0;
        typingAudio.play().catch(e => console.error('Error playing typing sound:', e));
      }
    } catch (error) {
      console.error('Error starting call:', error);
      setError('Failed to start call. Please try again.');
      toast.error('Failed to start call');
      errorBoundary.captureError(error);
    } finally {
      setLoading(false);
    }
  }, [calls, loading, toast, errorBoundary, typingAudio]);

  // Handle ending a call with transcript
  const handleEndCall = useCallback(async (transcript = '') => {
    if (loading || !activeCall) return;

    try {
      setLoading(true);
      setError(null);

      if (typingAudio) {
        typingAudio.pause();
      }

      await callService.updateCall(activeCall.id, {
        ...activeCall,
        callStatus: CALL_STATUS.COMPLETED,
        endTime: new Date(),
        transcript: transcript || 'Call completed successfully.'
      });

      const updatedCalls = calls.map(c =>
        c.id === activeCall.id
          ? {
              ...c,
              callStatus: CALL_STATUS.COMPLETED,
              endTime: new Date(),
              transcript: transcript || 'Call completed successfully.'
            }
          : c
      );

      setCalls(updatedCalls);
      setActiveCall(null);
      toast.success('Call ended successfully');
    } catch (error) {
      console.error('Error ending call:', error);
      setError('Failed to end call. Please try again.');
      toast.error('Failed to end call');
      errorBoundary.captureError(error);
    } finally {
      setLoading(false);
    }
  }, [calls, activeCall, loading, toast, errorBoundary, typingAudio]);

  // Render call list
  const renderCallList = useMemo(() => (
    <div className="mt-6 bg-white shadow overflow-hidden sm:rounded-md">
      <ul className="divide-y divide-gray-200">
        {calls.length > 0 ? (
          calls.map((call) => (
            <li key={call.id}>
              <div className="px-4 py-4 sm:px-6">
                <div className="flex items-center justify-between">
                  <div className="text-sm font-medium text-blue-600 truncate">
                    {call.callType === CALL_TYPES.OUTBOUND ? 'Outbound: ' : 'Inbound: '}
                    {call.callPurpose.replace('_', ' ')}
                  </div>
                  <div className="ml-2 flex-shrink-0 flex">
                    <span className={`px-2 inline-flex text-xs leading-5 font-semibold rounded-full ${
                      call.callStatus === CALL_STATUS.COMPLETED ? 'bg-green-100 text-green-800' :
                      call.callStatus === CALL_STATUS.CANCELLED ? 'bg-red-100 text-red-800' :
                      call.callStatus === CALL_STATUS.IN_PROGRESS ? 'bg-blue-100 text-blue-800' :
                      'bg-yellow-100 text-yellow-800'
                    }`}>
                      {call.callStatus.replace('_', ' ')}
                    </span>
                  </div>
                </div>
                <div className="mt-2 sm:flex sm:justify-between">
                  <div className="sm:flex">
                    <div className="flex items-center text-sm text-gray-500">
                      <svg className="flex-shrink-0 mr-1.5 h-5 w-5 text-gray-400" fill="none" viewBox="0 0 24 24" stroke="currentColor">
                        <path strokeLinecap="round" strokeLinejoin="round" strokeWidth="2" d="M16 7a4 4 0 11-8 0 4 4 0 018 0zM12 14a7 7 0 00-7 7h14a7 7 0 00-7-7z" />
                      </svg>
                      <span>{getPatientName(call.patientId)}</span>
                    </div>
                  </div>
                  <div className="mt-2 flex items-center text-sm text-gray-500 sm:mt-0">
                    <svg className="flex-shrink-0 mr-1.5 h-5 w-5 text-gray-400" fill="none" viewBox="0 0 24 24" stroke="currentColor">
                      <path strokeLinecap="round" strokeLinejoin="round" strokeWidth="2" d="M8 7V3m8 4V3m-9 8h10M5 21h14a2 2 0 002-2V7a2 2 0 00-2-2H5a2 2 0 00-2 2v12a2 2 0 002 2z" />
                    </svg>
                    <span>
                      {format(new Date(call.scheduledTime), 'MMM d, yyyy')} at {format(new Date(call.scheduledTime), 'h:mm a')}
                    </span>
                  </div>
                </div>
                {call.notes && (
                  <div className="mt-2">
                    <p className="text-sm text-gray-500">{call.notes}</p>
                  </div>
                )}
                {call.callStatus === CALL_STATUS.SCHEDULED && (
                  <div className="mt-3">
                    <button
                      onClick={() => handleStartCall(call)}
                      className="inline-flex items-center px-3 py-1 border border-transparent text-xs font-medium rounded-md shadow-sm text-white bg-green-600 hover:bg-green-700 focus:outline-none focus:ring-2 focus:ring-offset-2 focus:ring-green-500"
                      disabled={loading}
                    >
                      <svg className="-ml-0.5 mr-1 h-4 w-4" fill="none" viewBox="0 0 24 24" stroke="currentColor">
                        <path strokeLinecap="round" strokeLinejoin="round" strokeWidth="2" d="M3 5a2 2 0 012-2h3.28a1 1 0 01.948.684l1.498 4.493a1 1 0 01-.502 1.21l-2.257 1.13a11.042 11.042 0 005.516 5.516l1.13-2.257a1 1 0 011.21-.502l4.493 1.498a1 1 0 01.684.949V19a2 2 0 01-2 2h-1C9.716 21 3 14.284 3 6V5z" />
                      </svg>
                      Start Call
                    </button>
                  </div>
                )}
              </div>
            </li>
          ))
        ) : (
          <li className="px-4 py-4 sm:px-6 text-sm text-gray-500">
            No calls found.
          </li>
        )}
      </ul>
    </div>
  ), [calls, loading, getPatientName, handleStartCall]);

  if (loading && calls.length === 0) {
    return (
      <div className="flex items-center justify-center h-64">
        <div className="animate-spin rounded-full h-12 w-12 border-t-2 border-b-2 border-blue-500"></div>
      </div>
    );
  }

  return (
    <div>
      <div className="flex justify-between items-center">
        <h1 className="text-2xl font-semibold text-gray-900">Calls</h1>
        <button
          onClick={() => setShowAddModal(true)}
          className="inline-flex items-center px-4 py-2 border border-transparent text-sm font-medium rounded-md shadow-sm text-white bg-blue-600 hover:bg-blue-700 focus:outline-none focus:ring-2 focus:ring-offset-2 focus:ring-blue-500"
          aria-label="Schedule new call"
        >
          <svg className="-ml-1 mr-2 h-5 w-5" fill="none" viewBox="0 0 24 24" stroke="currentColor">
            <path strokeLinecap="round" strokeLinejoin="round" strokeWidth="2" d="M12 6v6m0 0v6m0-6h6m-6 0H6" />
          </svg>
          Schedule Call
        </button>
      </div>

      {error && (
        <div className="mt-4 bg-red-100 border border-red-400 text-red-700 px-4 py-3 rounded relative" role="alert">
          <span className="block sm:inline">{error}</span>
        </div>
      )}

      {renderCallList()}

      {showAddModal && (
        <div className="fixed z-10 inset-0 overflow-y-auto" role="dialog" aria-modal="true" aria-labelledby="modal-title">
          <div className="flex items-end justify-center min-h-screen pt-4 px-4 pb-20 text-center sm:block sm:p-0">
            <div className="fixed inset-0 bg-gray-500 bg-opacity-75 transition-opacity" aria-hidden="true" onClick={() => setShowAddModal(false)}></div>

            <div className="inline-block align-bottom bg-white rounded-lg text-left overflow-hidden shadow-xl transform transition-all sm:my-8 sm:align-middle sm:max-w-lg sm:w-full">
              <form onSubmit={handleSubmit} ref={formRef}>
                <div className="bg-white px-4 pt-5 pb-4 sm:p-6 sm:pb-4">
                  <div className="sm:flex sm:items-start">
                    <div className="mt-3 text-center sm:mt-0 sm:ml-4 sm:text-left w-full">
                      <h3 className="text-lg leading-6 font-medium text-gray-900" id="modal-title">
                        Schedule New Call
                      </h3>
                      <div className="mt-4 grid grid-cols-1 gap-y-6 gap-x-4 sm:grid-cols-6">
                        <div className="sm:col-span-6">
                          <label htmlFor="patientId" className="block text-sm font-medium text-gray-700">
                            Patient
                          </label>
                          <div className="mt-1">
                            <select
                              id="patientId"
                              name="patientId"
                              value={newCall.patientId}
                              onChange={handleInputChange}
                              className="shadow-sm focus:ring-blue-500 focus:border-blue-500 block w-full sm:text-sm border-gray-300 rounded-md"
                              required
                            >
                              <option value="">Select a patient</option>
                              {patients.map(patient => (
                                <option key={patient.id} value={patient.id}>
                                  {`${patient.firstName} ${patient.lastName}`}
                                </option>
                              ))}
                            </select>
                            {formErrors.patientId && (
                              <p className="mt-1 text-sm text-red-600" id="patientId-error">
                                {formErrors.patientId}
                              </p>
                            )}
                          </div>
                        </div>

                        <div className="sm:col-span-6">
                          <label htmlFor="callType" className="block text-sm font-medium text-gray-700">
                            Call Type
                          </label>
                          <div className="mt-1">
                            <select
                              id="callType"
                              name="callType"
                              value={newCall.callType}
                              onChange={handleInputChange}
                              className="shadow-sm focus:ring-blue-500 focus:border-blue-500 block w-full sm:text-sm border-gray-300 rounded-md"
                              required
                            >
                              {Object.values(CALL_TYPES).map(type => (
                                <option key={type} value={type}>
                                  {type.replace('_', ' ')}
                                </option>
                              ))}
                            </select>
                          </div>
                        </div>

                        <div className="sm:col-span-6">
                          <label htmlFor="callPurpose" className="block text-sm font-medium text-gray-700">
                            Call Purpose
                          </label>
                          <div className="mt-1">
                            <select
                              id="callPurpose"
                              name="callPurpose"
                              value={newCall.callPurpose}
                              onChange={handleInputChange}
                              className="shadow-sm focus:ring-blue-500 focus:border-blue-500 block w-full sm:text-sm border-gray-300 rounded-md"
                              required
                            >
                              {Object.values(CALL_PURPOSES).map(purpose => (
                                <option key={purpose} value={purpose}>
                                  {purpose.replace('_', ' ')}
                                </option>
                              ))}
                            </select>
                          </div>
                        </div>

                        <div className="sm:col-span-6">
                          <label htmlFor="scheduledTime" className="block text-sm font-medium text-gray-700">
                            Scheduled Date
                          </label>
                          <div className="mt-1">
                            <input
                              type="date"
                              id="scheduledTime"
                              name="scheduledTime"
                              value={newCall.scheduledTime}
                              onChange={handleInputChange}
                              className="shadow-sm focus:ring-blue-500 focus:border-blue-500 block w-full sm:text-sm border-gray-300 rounded-md"
                              required
                            />
                            {formErrors.scheduledTime && (
                              <p className="mt-1 text-sm text-red-600" id="scheduledTime-error">
                                {formErrors.scheduledTime}
                              </p>
                            )}
                          </div>
                        </div>

                        <div className="sm:col-span-3">
                          <label htmlFor="scheduledTimeHour" className="block text-sm font-medium text-gray-700">
                            Hour
                          </label>
                          <div className="mt-1">
                            <select
                              id="scheduledTimeHour"
                              name="scheduledTimeHour"
                              value={newCall.scheduledTimeHour}
                              onChange={handleInputChange}
                              className="shadow-sm focus:ring-blue-500 focus:border-blue-500 block w-full sm:text-sm border-gray-300 rounded-md"
                              required
                            >
                              {Array.from({ length: 24 }, (_, i) => i.toString().padStart(2, '0')).map(hour => (
                                <option key={hour} value={hour}>
                                  {hour}
                                </option>
                              ))}
                            </select>
                          </div>
                        </div>

                        <div className="sm:col-span-3">
                          <label htmlFor="scheduledTimeMinute" className="block text-sm font-medium text-gray-700">
                            Minute
                          </label>
                          <div className="mt-1">
                            <select
                              id="scheduledTimeMinute"
                              name="scheduledTimeMinute"
                              value={newCall.scheduledTimeMinute}
                              onChange={handleInputChange}
                              className="shadow-sm focus:ring-blue-500 focus:border-blue-500 block w-full sm:text-sm border-gray-300 rounded-md"
                              required
                            >
                              {Array.from({ length: 6 }, (_, i) => (i * 10).toString().padStart(2, '0')).map(minute => (
                                <option key={minute} value={minute}>
                                  {minute}
                                </option>
                              ))}
                            </select>
                          </div>
                        </div>

                        <div className="sm:col-span-6">
                          <label htmlFor="notes" className="block text-sm font-medium text-gray-700">
                            Notes
                          </label>
                          <div className="mt-1">
                            <textarea
                              id="notes"
                              name="notes"
                              rows={3}
                              value={newCall.notes}
                              onChange={handleInputChange}
                              className="shadow-sm focus:ring-blue-500 focus:border-blue-500 block w-full sm:text-sm border-gray-300 rounded-md"
                            />
                          </div>
                        </div>
                      </div>
                    </div>
                  </div>
                </div>
                <div className="bg-gray-50 px-4 py-3 sm:px-6 sm:flex sm:flex-row-reverse">
                  <button
                    type="submit"
                    disabled={loading}
                    className="inline-flex justify-center py-2 px-4 border border-transparent shadow-sm text-sm font-medium rounded-md text-white bg-blue-600 hover:bg-blue-700 focus:outline-none focus:ring-2 focus:ring-offset-2 focus:ring-blue-500"
                  >
                    {loading ? 'Scheduling...' : 'Schedule Call'}
                  </button>
                  <button
                    type="button"
                    onClick={() => setShowAddModal(false)}
                    className="mt-3 w-full inline-flex justify-center rounded-md border border-gray-300 shadow-sm px-4 py-2 bg-white text-base font-medium text-gray-700 hover:bg-gray-50 focus:outline-none focus:ring-2 focus:ring-offset-2 focus:ring-blue-500 sm:mt-0 sm:ml-3 sm:w-auto sm:text-sm"
                  >
                    Cancel
                  </button>
                </div>
              </form>
            </div>
          </div>
        </div>
      )}

      {/* Active Call UI */}
      {activeCall && (
        <div className="fixed inset-0 z-50 overflow-y-auto bg-gray-900 bg-opacity-75 flex items-center justify-center" role="dialog" aria-modal="true" aria-labelledby="active-call-title">
          <div className="bg-white rounded-lg shadow-xl max-w-2xl w-full mx-4">
            <div className="px-6 py-4 border-b border-gray-200">
              <h3 className="text-lg font-medium text-gray-900" id="active-call-title">
                {activeCall.callType === CALL_TYPES.OUTBOUND ? 'Outbound Call' : 'Inbound Call'}: {getPatientName(activeCall.patientId)}
              </h3>
              <p className="text-sm text-gray-500">{activeCall.callPurpose.replace('_', ' ')}</p>
            </div>
            <div className="px-6 py-4">
              <div className="mb-4">
                <div className="flex items-center justify-center mb-4">
                  <div className="h-20 w-20 rounded-full bg-blue-100 flex items-center justify-center">
                    <svg className="h-10 w-10 text-blue-600" fill="none" viewBox="0 0 24 24" stroke="currentColor">
                      <path strokeLinecap="round" strokeLinejoin="round" strokeWidth="2" d="M3 5a2 2 0 012-2h3.28a1 1 0 01.948.684l1.498 4.493a1 1 0 01-.502 1.21l-2.257 1.13a11.042 11.042 0 005.516 5.516l1.13-2.257a1 1 0 011.21-.502l4.493 1.498a1 1 0 01.684.949V19a2 2 0 01-2 2h-1C9.716 21 3 14.284 3 6V5z" />
                    </svg>
                  </div>
                </div>
                <div className="text-center mb-4">
                  <p className="text-lg font-medium">Call in progress...</p>
                  <p className="text-sm text-gray-500">
                    Started at {format(new Date(activeCall.startTime), 'h:mm a')}
                  </p>
                </div>
                <div className="bg-gray-100 p-4 rounded-lg mb-4 min-h-[100px] relative">
                  <div className="typing-indicator absolute bottom-4 left-4">
                    <span className="dot"></span>
                    <span className="dot"></span>
                    <span className="dot"></span>
                  </div>
                </div>
              </div>
              <div className="flex justify-end">
                <button
                  onClick={() => handleEndCall()}
                  className="inline-flex items-center px-4 py-2 border border-transparent text-sm font-medium rounded-md shadow-sm text-white bg-red-600 hover:bg-red-700 focus:outline-none focus:ring-2 focus:ring-offset-2 focus:ring-red-500"
                  disabled={loading}
                >
                  <svg className="-ml-1 mr-2 h-5 w-5" fill="none" viewBox="0 0 24 24" stroke="currentColor">
                    <path strokeLinecap="round" strokeLinejoin="round" strokeWidth="2" d="M6 18L18 6M6 6l12 12" />
                  </svg>
                  {loading ? 'Ending...' : 'End Call'}
                </button>
              </div>
            </div>
          </div>
        </div>
      )}
    </div>
  );
};

export default Calls;

In [ ]:
import React, { useState, useEffect } from 'react';
import { useApi } from '../../services/api';
import { patientService, appointmentService, leadService, callService } from '../../firebase/firestore';

const Dashboard = () => {
  const [stats, setStats] = useState({
    patients: 0,
    appointments: 0,
    leads: 0,
    calls: 0
  });
  const [recentAppointments, setRecentAppointments] = useState([]);
  const [upcomingCalls, setUpcomingCalls] = useState([]);
  const [loading, setLoading] = useState(true);
  const [error, setError] = useState(null);

  // Use API context for backend calls
  const api = useApi();

  useEffect(() => {
    const fetchDashboardData = async () => {
      try {
        setLoading(true);
        setError(null);

        // Fetch data from Firestore
        const [patients, appointments, leads, calls] = await Promise.all([
          patientService.getAllPatients(),
          appointmentService.getAllAppointments(),
          leadService.getAllLeads(),
          callService.getAllCalls()
        ]);

        // Update stats
        setStats({
          patients: patients.length,
          appointments: appointments.length,
          leads: leads.length,
          calls: calls.length
        });

        // Get recent appointments (next 5 upcoming)
        const now = new Date();
        const upcoming = appointments
          .filter(apt => apt.appointmentDate > now)
          .sort((a, b) => a.appointmentDate - b.appointmentDate)
          .slice(0, 5);

        setRecentAppointments(upcoming);

        // Get upcoming calls (next 5)
        const pendingCalls = calls
          .filter(call => call.callStatus !== 'completed' && call.scheduledTime > now)
          .sort((a, b) => a.scheduledTime - b.scheduledTime)
          .slice(0, 5);

        setUpcomingCalls(pendingCalls);

        setLoading(false);
      } catch (error) {
        console.error('Error fetching dashboard data:', error);
        setError('Failed to load dashboard data. Please try again.');
        setLoading(false);
      }
    };

    fetchDashboardData();
  }, []);

  if (loading) {
    return (
      <div className="flex items-center justify-center h-64">
        <div className="animate-spin rounded-full h-12 w-12 border-t-2 border-b-2 border-blue-500"></div>
      </div>
    );
  }

  if (error) {
    return (
      <div className="bg-red-100 border border-red-400 text-red-700 px-4 py-3 rounded relative" role="alert">
        <span className="block sm:inline">{error}</span>
      </div>
    );
  }

  return (
    <div>
      <h1 className="text-2xl font-semibold text-gray-900">Dashboard</h1>

      {/* Stats Cards */}
      <div className="mt-6 grid grid-cols-1 gap-5 sm:grid-cols-2 lg:grid-cols-4">
        <div className="bg-white overflow-hidden shadow rounded-lg">
          <div className="px-4 py-5 sm:p-6">
            <div className="flex items-center">
              <div className="flex-shrink-0 bg-blue-500 rounded-md p-3">
                <svg className="h-6 w-6 text-white" fill="none" viewBox="0 0 24 24" stroke="currentColor">
                  <path strokeLinecap="round" strokeLinejoin="round" strokeWidth="2" d="M17 20h5v-2a3 3 0 00-5.356-1.857M17 20H7m10 0v-2c0-.656-.126-1.283-.356-1.857M7 20H2v-2a3 3 0 015.356-1.857M7 20v-2c0-.656.126-1.283.356-1.857m0 0a5.002 5.002 0 019.288 0M15 7a3 3 0 11-6 0 3 3 0 016 0zm6 3a2 2 0 11-4 0 2 2 0 014 0zM7 10a2 2 0 11-4 0 2 2 0 014 0z" />
                </svg>
              </div>
              <div className="ml-5 w-0 flex-1">
                <dl>
                  <dt className="text-sm font-medium text-gray-500 truncate">
                    Total Patients
                  </dt>
                  <dd>
                    <div className="text-lg font-medium text-gray-900">
                      {stats.patients}
                    </div>
                  </dd>
                </dl>
              </div>
            </div>
          </div>
        </div>

        <div className="bg-white overflow-hidden shadow rounded-lg">
          <div className="px-4 py-5 sm:p-6">
            <div className="flex items-center">
              <div className="flex-shrink-0 bg-green-500 rounded-md p-3">
                <svg className="h-6 w-6 text-white" fill="none" viewBox="0 0 24 24" stroke="currentColor">
                  <path strokeLinecap="round" strokeLinejoin="round" strokeWidth="2" d="M8 7V3m8 4V3m-9 8h10M5 21h14a2 2 0 002-2V7a2 2 0 00-2-2H5a2 2 0 00-2 2v12a2 2 0 002 2z" />
                </svg>
              </div>
              <div className="ml-5 w-0 flex-1">
                <dl>
                  <dt className="text-sm font-medium text-gray-500 truncate">
                    Appointments
                  </dt>
                  <dd>
                    <div className="text-lg font-medium text-gray-900">
                      {stats.appointments}
                    </div>
                  </dd>
                </dl>
              </div>
            </div>
          </div>
        </div>

        <div className="bg-white overflow-hidden shadow rounded-lg">
          <div className="px-4 py-5 sm:p-6">
            <div className="flex items-center">
              <div className="flex-shrink-0 bg-yellow-500 rounded-md p-3">
                <svg className="h-6 w-6 text-white" fill="none" viewBox="0 0 24 24" stroke="currentColor">
                  <path strokeLinecap="round" strokeLinejoin="round" strokeWidth="2" d="M9 19v-6a2 2 0 00-2-2H5a2 2 0 00-2 2v6a2 2 0 002 2h2a2 2 0 002-2zm0 0V9a2 2 0 012-2h2a2 2 0 012 2v10m-6 0a2 2 0 002 2h2a2 2 0 002-2m0 0V5a2 2 0 012-2h2a2 2 0 012 2v14a2 2 0 01-2 2h-2a2 2 0 01-2-2z" />
                </svg>
              </div>
              <div className="ml-5 w-0 flex-1">
                <dl>
                  <dt className="text-sm font-medium text-gray-500 truncate">
                    Leads
                  </dt>
                  <dd>
                    <div className="text-lg font-medium text-gray-900">
                      {stats.leads}
                    </div>
                  </dd>
                </dl>
              </div>
            </div>
          </div>
        </div>

        <div className="bg-white overflow-hidden shadow rounded-lg">
          <div className="px-4 py-5 sm:p-6">
            <div className="flex items-center">
              <div className="flex-shrink-0 bg-purple-500 rounded-md p-3">
                <svg className="h-6 w-6 text-white" fill="none" viewBox="0 0 24 24" stroke="currentColor">
                  <path strokeLinecap="round" strokeLinejoin="round" strokeWidth="2" d="M3 5a2 2 0 012-2h3.28a1 1 0 01.948.684l1.498 4.493a1 1 0 01-.502 1.21l-2.257 1.13a11.042 11.042 0 005.516 5.516l1.13-2.257a1 1 0 011.21-.502l4.493 1.498a1 1 0 01.684.949V19a2 2 0 01-2 2h-1C9.716 21 3 14.284 3 6V5z" />
                </svg>
              </div>
              <div className="ml-5 w-0 flex-1">
                <dl>
                  <dt className="text-sm font-medium text-gray-500 truncate">
                    Calls
                  </dt>
                  <dd>
                    <div className="text-lg font-medium text-gray-900">
                      {stats.calls}
                    </div>
                  </dd>
                </dl>
              </div>
            </div>
          </div>
        </div>
      </div>

      {/* Recent Appointments */}
      <div className="mt-8">
        <h2 className="text-lg font-medium text-gray-900">Upcoming Appointments</h2>
        <div className="mt-4 bg-white shadow overflow-hidden sm:rounded-md">
          <ul className="divide-y divide-gray-200">
            {recentAppointments.length > 0 ? (
              recentAppointments.map((appointment) => (
                <li key={appointment.id}>
                  <div className="px-4 py-4 sm:px-6">
                    <div className="flex items-center justify-between">
                      <div className="text-sm font-medium text-blue-600 truncate">
                        {appointment.appointmentType}
                      </div>
                      <div className="ml-2 flex-shrink-0 flex">
                        <span className={`px-2 inline-flex text-xs leading-5 font-semibold rounded-full ${
                          appointment.status === 'scheduled' ? 'bg-green-100 text-green-800' :
                          appointment.status === 'cancelled' ? 'bg-red-100 text-red-800' :
                          'bg-yellow-100 text-yellow-800'
                        }`}>
                          {appointment.status}
                        </span>
                      </div>
                    </div>
                    <div className="mt-2 sm:flex sm:justify-between">
                      <div className="sm:flex">
                        <div className="flex items-center text-sm text-gray-500">
                          <svg className="flex-shrink-0 mr-1.5 h-5 w-5 text-gray-400" fill="none" viewBox="0 0 24 24" stroke="currentColor">
                            <path strokeLinecap="round" strokeLinejoin="round" strokeWidth="2" d="M16 7a4 4 0 11-8 0 4 4 0 018 0zM12 14a7 7 0 00-7 7h14a7 7 0 00-7-7z" />
                          </svg>
                          <span>Patient ID: {appointment.patientId}</span>
                        </div>
                      </div>
                      <div className="mt-2 flex items-center text-sm text-gray-500 sm:mt-0">
                        <svg className="flex-shrink-0 mr-1.5 h-5 w-5 text-gray-400" fill="none" viewBox="0 0 24 24" stroke="currentColor">
                          <path strokeLinecap="round" strokeLinejoin="round" strokeWidth="2" d="M8 7V3m8 4V3m-9 8h10M5 21h14a2 2 0 002-2V7a2 2 0 00-2-2H5a2 2 0 00-2 2v12a2 2 0 002 2z" />
                        </svg>
                        <span>
                          {appointment.appointmentDate.toLocaleDateString()} at {appointment.appointmentDate.toLocaleTimeString([], { hour: '2-digit', minute: '2-digit' })}
                        </span>
                      </div>
                    </div>
                  </div>
                </li>
              ))
            ) : (
              <li className="px-4 py-4 sm:px-6 text-sm text-gray-500">
                No upcoming appointments.
              </li>
            )}
          </ul>
        </div>
      </div>

      {/* Upcoming Calls */}
      <div className="mt-8">
        <h2 className="text-lg font-medium text-gray-900">Upcoming Calls</h2>
        <div className="mt-4 bg-white shadow overflow-hidden sm:rounded-md">
          <ul className="divide-y divide-gray-200">
            {upcomingCalls.length > 0 ? (
              upcomingCalls.map((call) => (
                <li key={call.id}>
                  <div className="px-4 py-4 sm:px-6">
                    <div className="flex items-center justify-between">
                      <div className="text-sm font-medium text-blue-600 truncate">
                        {call.callType} - {call.callPurpose}
                      </div>
                      <div className="ml-2 flex-shrink-0 flex">
                        <span className={`px-2 inline-flex text-xs leading-5 font-semibold rounded-full ${
                          call.callStatus === 'scheduled' ? 'bg-green-100 text-green-800' :
                          call.callStatus === 'cancelled' ? 'bg-red-100 text-red-800' :
                          'bg-yellow-100 text-yellow-800'
                        }`}>
                          {call.callStatus}
                        </span>
                      </div>
                    </div>
                    <div className="mt-2 sm:flex sm:justify-between">
                      <div className="sm:flex">
                        <div className="flex items-center text-sm text-gray-500">
                          <svg className="flex-shrink-0 mr-1.5 h-5 w-5 text-gray-400" fill="none" viewBox="0 0 24 24" stroke="currentColor">
                            <path strokeLinecap="round" strokeLinejoin="round" strokeWidth="2" d="M16 7a4 4 0 11-8 0 4 4 0 018 0zM12 14a7 7 0 00-7 7h14a7 7 0 00-7-7z" />
                          </svg>
                          <span>Patient ID: {call.patientId}</span>
                        </div>
                      </div>
                      <div className="mt-2 flex items-center text-sm text-gray-500 sm:mt-0">
                        <svg className="flex-shrink-0 mr-1.5 h-5 w-5 text-gray-400" fill="none" viewBox="0 0 24 24" stroke="currentColor">
                          <path strokeLinecap="round" strokeLinejoin="round" strokeWidth="2" d="M3 5a2 2 0 012-2h3.28a1 1 0 01.948.684l1.498 4.493a1 1 0 01-.502 1.21l-2.257 1.13a11.042 11.042 0 005.516 5.516l1.13-2.257a1 1 0 011.21-.502l4.493 1.498a1 1 0 01.684.949V19a2 2 0 01-2 2h-1C9.716 21 3 14.284 3 6V5z" />
                        </svg>
                        <span>
                          {call.scheduledTime.toLocaleDateString()} at {call.scheduledTime.toLocaleTimeString([], { hour: '2-digit', minute: '2-digit' })}
                        </span>
                      </div>
                    </div>
                  </div>
                </li>
              ))
            ) : (
              <li className="px-4 py-4 sm:px-6 text-sm text-gray-500">
                No upcoming calls.
              </li>
            )}
          </ul>
        </div>
      </div>
    </div>
  );
};

export default Dashboard;

In [ ]:
// Firebase configuration and initialization
import { initializeApp } from 'firebase/app';
import { getAuth } from 'firebase/auth';
import { getFirestore } from 'firebase/firestore';

// Firebase configuration
const firebaseConfig = {
  apiKey: process.env.REACT_APP_FIREBASE_API_KEY,
  authDomain: process.env.REACT_APP_FIREBASE_AUTH_DOMAIN,
  projectId: process.env.REACT_APP_FIREBASE_PROJECT_ID,
  storageBucket: process.env.REACT_APP_FIREBASE_STORAGE_BUCKET,
  messagingSenderId: process.env.REACT_APP_FIREBASE_MESSAGING_SENDER_ID,
  appId: process.env.REACT_APP_FIREBASE_APP_ID
};

// Initialize Firebase
const app = initializeApp(firebaseConfig);

// Initialize Firebase services
export const auth = getAuth(app);
export const db = getFirestore(app);

export default app;

In [ ]:
import {
  collection,
  doc,
  getDoc,
  getDocs,
  addDoc,
  updateDoc,
  deleteDoc,
  query,
  where,
  orderBy,
  serverTimestamp
} from 'firebase/firestore';
import { db } from './firebase';

// Patient service
export const patientService = {
  // Get all patients
  getAllPatients: async () => {
    try {
      const q = query(collection(db, 'patients'), orderBy('lastName'), orderBy('firstName'));
      const snapshot = await getDocs(q);
      return snapshot.docs.map(doc => ({
        id: doc.id,
        ...doc.data()
      }));
    } catch (error) {
      console.error('Error getting patients:', error);
      throw error;
    }
  },

  // Get patient by ID
  getPatient: async (id) => {
    try {
      const docRef = doc(db, 'patients', id);
      const docSnap = await getDoc(docRef);

      if (docSnap.exists()) {
        return {
          id: docSnap.id,
          ...docSnap.data()
        };
      }

      return null;
    } catch (error) {
      console.error('Error getting patient:', error);
      throw error;
    }
  },

  // Add new patient
  addPatient: async (patientData) => {
    try {
      const docRef = await addDoc(collection(db, 'patients'), {
        ...patientData,
        createdAt: serverTimestamp()
      });

      return docRef.id;
    } catch (error) {
      console.error('Error adding patient:', error);
      throw error;
    }
  },

  // Update patient
  updatePatient: async (id, patientData) => {
    try {
      const docRef = doc(db, 'patients', id);
      await updateDoc(docRef, {
        ...patientData,
        updatedAt: serverTimestamp()
      });

      return true;
    } catch (error) {
      console.error('Error updating patient:', error);
      throw error;
    }
  },

  // Delete patient
  deletePatient: async (id) => {
    try {
      const docRef = doc(db, 'patients', id);
      await deleteDoc(docRef);

      return true;
    } catch (error) {
      console.error('Error deleting patient:', error);
      throw error;
    }
  }
};

// Appointment service
export const appointmentService = {
  // Get all appointments
  getAllAppointments: async () => {
    try {
      const q = query(collection(db, 'appointments'), orderBy('appointmentDate', 'desc'));
      const snapshot = await getDocs(q);
      return snapshot.docs.map(doc => ({
        id: doc.id,
        ...doc.data(),
        appointmentDate: doc.data().appointmentDate?.toDate()
      }));
    } catch (error) {
      console.error('Error getting appointments:', error);
      throw error;
    }
  },

  // Get appointments for a patient
  getPatientAppointments: async (patientId) => {
    try {
      const q = query(
        collection(db, 'appointments'),
        where('patientId', '==', patientId),
        orderBy('appointmentDate', 'desc')
      );
      const snapshot = await getDocs(q);
      return snapshot.docs.map(doc => ({
        id: doc.id,
        ...doc.data(),
        appointmentDate: doc.data().appointmentDate?.toDate()
      }));
    } catch (error) {
      console.error('Error getting patient appointments:', error);
      throw error;
    }
  },

  // Get appointment by ID
  getAppointment: async (id) => {
    try {
      const docRef = doc(db, 'appointments', id);
      const docSnap = await getDoc(docRef);

      if (docSnap.exists()) {
        const data = docSnap.data();
        return {
          id: docSnap.id,
          ...data,
          appointmentDate: data.appointmentDate?.toDate()
        };
      }

      return null;
    } catch (error) {
      console.error('Error getting appointment:', error);
      throw error;
    }
  },

  // Add new appointment
  addAppointment: async (appointmentData) => {
    try {
      const docRef = await addDoc(collection(db, 'appointments'), {
        ...appointmentData,
        createdAt: serverTimestamp()
      });

      return docRef.id;
    } catch (error) {
      console.error('Error adding appointment:', error);
      throw error;
    }
  },

  // Update appointment
  updateAppointment: async (id, appointmentData) => {
    try {
      const docRef = doc(db, 'appointments', id);
      await updateDoc(docRef, {
        ...appointmentData,
        updatedAt: serverTimestamp()
      });

      return true;
    } catch (error) {
      console.error('Error updating appointment:', error);
      throw error;
    }
  },

  // Delete appointment
  deleteAppointment: async (id) => {
    try {
      const docRef = doc(db, 'appointments', id);
      await deleteDoc(docRef);

      return true;
    } catch (error) {
      console.error('Error deleting appointment:', error);
      throw error;
    }
  }
};

// Lead service
export const leadService = {
  // Get all leads
  getAllLeads: async () => {
    try {
      const q = query(collection(db, 'leads'), orderBy('createdAt', 'desc'));
      const snapshot = await getDocs(q);
      return snapshot.docs.map(doc => ({
        id: doc.id,
        ...doc.data(),
        createdAt: doc.data().createdAt?.toDate(),
        updatedAt: doc.data().updatedAt?.toDate()
      }));
    } catch (error) {
      console.error('Error getting leads:', error);
      throw error;
    }
  },

  // Get leads for a patient
  getLeadsByPatient: async (patientId) => {
    try {
      const q = query(
        collection(db, 'leads'),
        where('patientId', '==', patientId),
        orderBy('createdAt', 'desc')
      );
      const snapshot = await getDocs(q);
      return snapshot.docs.map(doc => ({
        id: doc.id,
        ...doc.data(),
        createdAt: doc.data().createdAt?.toDate(),
        updatedAt: doc.data().updatedAt?.toDate()
      }));
    } catch (error) {
      console.error('Error getting patient leads:', error);
      throw error;
    }
  },

  // Get lead by ID
  getLead: async (id) => {
    try {
      const docRef = doc(db, 'leads', id);
      const docSnap = await getDoc(docRef);

      if (docSnap.exists()) {
        const data = docSnap.data();
        return {
          id: docSnap.id,
          ...data,
          createdAt: data.createdAt?.toDate(),
          updatedAt: data.updatedAt?.toDate()
        };
      }

      return null;
    } catch (error) {
      console.error('Error getting lead:', error);
      throw error;
    }
  },

  // Add new lead
  addLead: async (leadData) => {
    try {
      const docRef = await addDoc(collection(db, 'leads'), {
        ...leadData,
        createdAt: serverTimestamp()
      });

      return docRef.id;
    } catch (error) {
      console.error('Error adding lead:', error);
      throw error;
    }
  },

  // Update lead
  updateLead: async (id, leadData) => {
    try {
      const docRef = doc(db, 'leads', id);
      await updateDoc(docRef, {
        ...leadData,
        updatedAt: serverTimestamp()
      });

      return true;
    } catch (error) {
      console.error('Error updating lead:', error);
      throw error;
    }
  },

  // Delete lead
  deleteLead: async (id) => {
    try {
      const docRef = doc(db, 'leads', id);
      await deleteDoc(docRef);

      return true;
    } catch (error) {
      console.error('Error deleting lead:', error);
      throw error;
    }
  }
};

// Call service
export const callService = {
  // Get all calls
  getAllCalls: async () => {
    try {
      const q = query(collection(db, 'calls'), orderBy('scheduledTime', 'desc'));
      const snapshot = await getDocs(q);
      return snapshot.docs.map(doc => ({
        id: doc.id,
        ...doc.data(),
        scheduledTime: doc.data().scheduledTime?.toDate(),
        completedAt: doc.data().completedAt?.toDate()
      }));
    } catch (error) {
      console.error('Error getting calls:', error);
      throw error;
    }
  },

  // Get calls for a patient
  getPatientCalls: async (patientId) => {
    try {
      const q = query(
        collection(db, 'calls'),
        where('patientId', '==', patientId),
        orderBy('scheduledTime', 'desc')
      );
      const snapshot = await getDocs(q);
      return snapshot.docs.map(doc => ({
        id: doc.id,
        ...doc.data(),
        scheduledTime: doc.data().scheduledTime?.toDate(),
        completedAt: doc.data().completedAt?.toDate()
      }));
    } catch (error) {
      console.error('Error getting patient calls:', error);
      throw error;
    }
  },

  // Get call by ID
  getCall: async (id) => {
    try {
      const docRef = doc(db, 'calls', id);
      const docSnap = await getDoc(docRef);

      if (docSnap.exists()) {
        const data = docSnap.data();
        return {
          id: docSnap.id,
          ...data,
          scheduledTime: data.scheduledTime?.toDate(),
          completedAt: data.completedAt?.toDate()
        };
      }

      return null;
    } catch (error) {
      console.error('Error getting call:', error);
      throw error;
    }
  },

  // Add new call
  addCall: async (callData) => {
    try {
      const docRef = await addDoc(collection(db, 'calls'), {
        ...callData,
        createdAt: serverTimestamp()
      });

      return docRef.id;
    } catch (error) {
      console.error('Error adding call:', error);
      throw error;
    }
  },

  // Update call
  updateCall: async (id, callData) => {
    try {
      const docRef = doc(db, 'calls', id);
      await updateDoc(docRef, {
        ...callData,
        updatedAt: serverTimestamp()
      });

      return true;
    } catch (error) {
      console.error('Error updating call:', error);
      throw error;
    }
  },

  // Complete call with transcript
  completeCall: async (id, transcript) => {
    try {
      const docRef = doc(db, 'calls', id);
      await updateDoc(docRef, {
        callStatus: 'completed',
        transcript,
        completedAt: serverTimestamp()
      });

      return true;
    } catch (error) {
      console.error('Error completing call:', error);
      throw error;
    }
  },

  // Delete call
  deleteCall: async (id) => {
    try {
      const docRef = doc(db, 'calls', id);
      await deleteDoc(docRef);

      return true;
    } catch (error) {
      console.error('Error deleting call:', error);
      throw error;
    }
  }
};


In [ ]:
require('dotenv').config();
const express = require('express');
const cors = require('cors');
const path = require('path');
const helmet = require('helmet');
const admin = require('firebase-admin');

// Initialize Firebase Admin SDK
if (process.env.FIREBASE_SERVICE_ACCOUNT) {
  try {
    const serviceAccount = JSON.parse(process.env.FIREBASE_SERVICE_ACCOUNT);
    admin.initializeApp({
      credential: admin.credential.cert(serviceAccount),
      databaseURL: process.env.FIREBASE_DATABASE_URL
    });
  } catch (error) {
    console.error('Error initializing Firebase Admin:', error);
  }
} else {
  console.warn('FIREBASE_SERVICE_ACCOUNT not found, Firebase Admin SDK not initialized');
}

const app = express();

// Security middleware
app.use(helmet({
  contentSecurityPolicy: {
    directives: {
      defaultSrc: ["'self'"],
      scriptSrc: ["'self'", "'unsafe-inline'", "https://www.gstatic.com", "https://*.firebaseio.com", "https://apis.google.com"],
      styleSrc: ["'self'", "'unsafe-inline'", "https://fonts.googleapis.com"],
      fontSrc: ["'self'", "https://fonts.gstatic.com"],
      imgSrc: ["'self'", "data:", "https://*.googleapis.com", "https://*.gstatic.com"],
      connectSrc: ["'self'", "https://*.firebaseio.com", "https://*.googleapis.com"],
      frameSrc: ["'self'", "https://*.firebaseapp.com"],
      objectSrc: ["'none'"]
    }
  }
}));

// Middleware
app.use(cors());
app.use(express.json());
app.use(express.urlencoded({ extended: false }));

// API routes
app.use('/api/health', (req, res) => {
  res.json({ status: 'ok', timestamp: new Date() });
});

// Import and use route modules
const authRoutes = require('./routes/auth');
const patientsRoutes = require('./routes/patients');
const appointmentsRoutes = require('./routes/appointments');
const leadsRoutes = require('./routes/leads');
const callsRoutes = require('./routes/calls');

app.use('/api/auth', authRoutes);
app.use('/api/patients', patientsRoutes);
app.use('/api/appointments', appointmentsRoutes);
app.use('/api/leads', leadsRoutes);
app.use('/api/calls', callsRoutes);

// Serve static assets in production
if (process.env.NODE_ENV === 'production') {
  // Set static folder
  app.use(express.static(path.join(__dirname, '../client/build')));

  app.get('*', (req, res) => {
    res.sendFile(path.resolve(__dirname, '../client/build', 'index.html'));
  });
}

// Error handling middleware
app.use((err, req, res, next) => {
  console.error(err.stack);
  res.status(500).json({
    error: 'Server error',
    message: process.env.NODE_ENV === 'production' ? 'An unexpected error occurred' : err.message
  });
});

const PORT = process.env.PORT || 5000;

app.listen(PORT, () => console.log(`Server running on port ${PORT}`));


In [ ]:
import React, { useState, useEffect, useCallback, useRef, useMemo } from 'react';
import { leadService, patientService } from '../../firebase/firestore';
import { useApi } from '../../services/api';
import { format, isValid, parseISO } from 'date-fns';
import { useToast } from '../../components/ui/ToastContext';
import { useErrorBoundary } from '../../components/ui/ErrorBoundary';
import { LEAD_SOURCES, LEAD_STATUSES, LEAD_SCORES } from '../../constants/leadTypes';

const LEAD_DEFAULTS = {
  patientId: '',
  source: LEAD_SOURCES.REFERRAL,
  status: LEAD_STATUSES.NEW,
  score: LEAD_SCORES.AVERAGE,
  notes: ''
};

const Leads = () => {
  const [leads, setLeads] = useState([]);
  const [patients, setPatients] = useState([]);
  const [loading, setLoading] = useState(true);
  const [error, setError] = useState(null);
  const [showAddModal, setShowAddModal] = useState(false);
  const [newLead, setNewLead] = useState(LEAD_DEFAULTS);
  const [formErrors, setFormErrors] = useState({});
  const toast = useToast();
  const api = useApi();
  const formRef = useRef(null);
  const errorBoundary = useErrorBoundary();

  // Initialize form validation
  useEffect(() => {
    if (formRef.current) {
      formRef.current.addEventListener('invalid', (e) => {
        e.preventDefault();
        setFormErrors(prev => ({
          ...prev,
          [e.target.name]: e.target.validationMessage
        }));
      });
    }

    return () => {
      if (formRef.current) {
        formRef.current.removeEventListener('invalid', () => {});
      }
    };
  }, []);

  // Fetch leads and patients with error handling
  const fetchData = useCallback(async () => {
    try {
      setLoading(true);
      setError(null);

      const [leadsData, patientsData] = await Promise.all([
        leadService.getAllLeads(),
        patientService.getAllPatients()
      ]);

      const sortedLeads = leadsData.sort((a, b) => b.score - a.score);

      setLeads(sortedLeads);
      setPatients(patientsData);
    } catch (error) {
      console.error('Error fetching data:', error);
      setError('Failed to load leads. Please try again.');
      toast.error('Failed to load leads');
      errorBoundary.captureError(error);
    } finally {
      setLoading(false);
    }
  }, [toast, errorBoundary]);

  useEffect(() => {
    fetchData();
    return () => {
      // Cleanup any subscriptions if needed
    };
  }, [fetchData]);

  // Validate form inputs
  const validateForm = useCallback(() => {
    const errors = {};

    if (!newLead.patientId) {
      errors.patientId = 'Please select a patient';
    }

    if (!Object.values(LEAD_SOURCES).includes(newLead.source)) {
      errors.source = 'Please select a valid lead source';
    }

    if (!Object.values(LEAD_STATUSES).includes(newLead.status)) {
      errors.status = 'Please select a valid status';
    }

    if (newLead.score < LEAD_SCORES.MIN || newLead.score > LEAD_SCORES.MAX) {
      errors.score = 'Score must be between 1 and 5';
    }

    return errors;
  }, [newLead]);

  // Handle form input changes with validation
  const handleInputChange = useCallback((e) => {
    const { name, value } = e.target;

    let newValue = value;
    if (name === 'score') {
      newValue = parseInt(value, 10);
      if (newValue < LEAD_SCORES.MIN) newValue = LEAD_SCORES.MIN;
      if (newValue > LEAD_SCORES.MAX) newValue = LEAD_SCORES.MAX;
    }

    setNewLead(prev => ({
      ...prev,
      [name]: newValue
    }));

    // Clear error for changed field
    setFormErrors(prev => ({
      ...prev,
      [name]: ''
    }));
  }, []);

  // Handle form submission with validation
  const handleSubmit = useCallback(async (e) => {
    e.preventDefault();

    const errors = validateForm();
    if (Object.keys(errors).length > 0) {
      setFormErrors(errors);
      return;
    }

    try {
      setLoading(true);
      setError(null);

      const leadData = {
        patientId: newLead.patientId,
        source: newLead.source,
        status: newLead.status,
        score: newLead.score,
        notes: newLead.notes,
        createdAt: new Date()
      };

      const leadId = await leadService.addLead(leadData);

      const patient = patients.find(p => p.id === newLead.patientId);

      setLeads(prev => [
        {
          id: leadId,
          ...leadData,
          patientName: patient ? `${patient.firstName} ${patient.lastName}` : 'Unknown Patient'
        },
        ...prev
      ].sort((a, b) => b.score - a.score));

      toast.success('Lead added successfully');

      // Reset form
      setNewLead(LEAD_DEFAULTS);
      setFormErrors({});
      setShowAddModal(false);
    } catch (error) {
      console.error('Error adding lead:', error);
      setError('Failed to add lead. Please try again.');
      toast.error('Failed to add lead');
      errorBoundary.captureError(error);
    } finally {
      setLoading(false);
    }
  }, [newLead, patients, validateForm, toast, errorBoundary]);

  // Get patient name by ID
  const getPatientName = useCallback((patientId) => {
    const patient = patients.find(p => p.id === patientId);
    return patient ? `${patient.firstName} ${patient.lastName}` : 'Unknown Patient';
  }, [patients]);

  // Get score color class
  const getScoreColorClass = useCallback((score) => {
    switch (score) {
      case LEAD_SCORES.EXCELLENT:
        return 'bg-green-100 text-green-800';
      case LEAD_SCORES.GOOD:
        return 'bg-blue-100 text-blue-800';
      case LEAD_SCORES.AVERAGE:
        return 'bg-yellow-100 text-yellow-800';
      case LEAD_SCORES.POOR:
        return 'bg-orange-100 text-orange-800';
      case LEAD_SCORES.POOR:
        return 'bg-red-100 text-red-800';
      default:
        return 'bg-gray-100 text-gray-800';
    }
  }, []);

  // Render leads list
  const renderLeadsList = useMemo(() => (
    <div className="mt-6 bg-white shadow overflow-hidden sm:rounded-md">
      <ul className="divide-y divide-gray-200">
        {leads.length > 0 ? (
          leads.map((lead) => (
            <li key={lead.id}>
              <div className="px-4 py-4 sm:px-6">
                <div className="flex items-center justify-between">
                  <div className="text-sm font-medium text-blue-600 truncate">
                    {lead.source.replace('_', ' ')}
                  </div>
                  <div className="ml-2 flex-shrink-0 flex">
                    <span className={`px-2 inline-flex text-xs leading-5 font-semibold rounded-full ${getScoreColorClass(lead.score)}`}>
                      Score: {lead.score}/5
                    </span>
                  </div>
                </div>
                <div className="mt-2 sm:flex sm:justify-between">
                  <div className="sm:flex">
                    <div className="flex items-center text-sm text-gray-500">
                      <svg className="flex-shrink-0 mr-1.5 h-5 w-5 text-gray-400" fill="none" viewBox="0 0 24 24" stroke="currentColor">
                        <path strokeLinecap="round" strokeLinejoin="round" strokeWidth="2" d="M16 7a4 4 0 11-8 0 4 4 0 018 0zM12 14a7 7 0 00-7 7h14a7 7 0 00-7-7z" />
                      </svg>
                      <span>{getPatientName(lead.patientId)}</span>
                    </div>
                    <div className="mt-2 flex items-center text-sm text-gray-500 sm:mt-0 sm:ml-6">
                      <svg className="flex-shrink-0 mr-1.5 h-5 w-5 text-gray-400" fill="none" viewBox="0 0 24 24" stroke="currentColor">
                        <path strokeLinecap="round" strokeLinejoin="round" strokeWidth="2" d="M9 12l2 2 4-4m6 2a9 9 0 11-18 0 9 9 0 0118 0z" />
                      </svg>
                      <span>{lead.status.replace('_', ' ')}</span>
                    </div>
                  </div>
                  <div className="mt-2 flex items-center text-sm text-gray-500 sm:mt-0">
                    <svg className="flex-shrink-0 mr-1.5 h-5 w-5 text-gray-400" fill="none" viewBox="0 0 24 24" stroke="currentColor">
                      <path strokeLinecap="round" strokeLinejoin="round" strokeWidth="2" d="M8 7V3m8 4V3m-9 8h10M5 21h14a2 2 0 002-2V7a2 2 0 00-2-2H5a2 2 0 00-2 2v12a2 2 0 002 2z" />
                    </svg>
                    <span>
                      {format(new Date(lead.createdAt), 'MMM d, yyyy')}
                    </span>
                  </div>
                </div>
                {lead.notes && (
                  <div className="mt-2">
                    <p className="text-sm text-gray-500">{lead.notes}</p>
                  </div>
                )}
              </div>
            </li>
          ))
        ) : (
          <li className="px-4 py-4 sm:px-6 text-sm text-gray-500">
            No leads found.
          </li>
        )}
      </ul>
    </div>
  ), [leads, getPatientName, getScoreColorClass]);

  if (loading && leads.length === 0) {
    return (
      <div className="flex items-center justify-center h-64">
        <div className="animate-spin rounded-full h-12 w-12 border-t-2 border-b-2 border-blue-500"></div>
      </div>
    );
  }

  return (
    <div>
      <div className="flex justify-between items-center">
        <h1 className="text-2xl font-semibold text-gray-900">Leads</h1>
        <button
          onClick={() => setShowAddModal(true)}
          className="inline-flex items-center px-4 py-2 border border-transparent text-sm font-medium rounded-md shadow-sm text-white bg-blue-600 hover:bg-blue-700 focus:outline-none focus:ring-2 focus:ring-offset-2 focus:ring-blue-500"
          aria-label="Add new lead"
        >
          <svg className="-ml-1 mr-2 h-5 w-5" fill="none" viewBox="0 0 24 24" stroke="currentColor">
            <path strokeLinecap="round" strokeLinejoin="round" strokeWidth="2" d="M12 6v6m0 0v6m0-6h6m-6 0H6" />
          </svg>
          Add Lead
        </button>
      </div>

      {error && (
        <div className="mt-4 bg-red-100 border border-red-400 text-red-700 px-4 py-3 rounded relative" role="alert">
          <span className="block sm:inline">{error}</span>
        </div>
      )}

      {renderLeadsList()}

      {showAddModal && (
        <div className="fixed z-10 inset-0 overflow-y-auto" role="dialog" aria-modal="true" aria-labelledby="modal-title">
          <div className="flex items-end justify-center min-h-screen pt-4 px-4 pb-20 text-center sm:block sm:p-0">
            <div className="fixed inset-0 bg-gray-500 bg-opacity-75 transition-opacity" aria-hidden="true" onClick={() => setShowAddModal(false)}></div>

            <div className="inline-block align-bottom bg-white rounded-lg text-left overflow-hidden shadow-xl transform transition-all sm:my-8 sm:align-middle sm:max-w-lg sm:w-full">
              <form onSubmit={handleSubmit} ref={formRef} noValidate>
                <div className="bg-white px-4 pt-5 pb-4 sm:p-6 sm:pb-4">
                  <div className="sm:flex sm:items-start">
                    <div className="mt-3 text-center sm:mt-0 sm:ml-4 sm:text-left w-full">
                      <h3 className="text-lg leading-6 font-medium text-gray-900" id="modal-title">
                        Add New Lead
                      </h3>
                      <div className="mt-4 grid grid-cols-1 gap-y-6 gap-x-4 sm:grid-cols-6">
                        <div className="sm:col-span-6">
                          <label htmlFor="patientId" className="block text-sm font-medium text-gray-700">
                            Patient
                          </label>
                          <div className="mt-1">
                            <select
                              id="patientId"
                              name="patientId"
                              required
                              value={newLead.patientId}
                              onChange={handleInputChange}
                              className="shadow-sm focus:ring-blue-500 focus:border-blue-500 block w-full sm:text-sm border-gray-300 rounded-md"
                            >
                              <option value="">Select a patient</option>
                              {patients.map(patient => (
                                <option key={patient.id} value={patient.id}>
                                  {patient.firstName} {patient.lastName}
                                </option>
                              ))}
                            </select>
                            {formErrors.patientId && (
                              <p className="mt-1 text-sm text-red-600" id="patientId-error">
                                {formErrors.patientId}
                              </p>
                            )}
                          </div>
                        </div>

                        <div className="sm:col-span-3">
                          <label htmlFor="source" className="block text-sm font-medium text-gray-700">
                            Lead Source
                          </label>
                          <div className="mt-1">
                            <select
                              id="source"
                              name="source"
                              required
                              value={newLead.source}
                              onChange={handleInputChange}
                              className="shadow-sm focus:ring-blue-500 focus:border-blue-500 block w-full sm:text-sm border-gray-300 rounded-md"
                            >
                              {Object.values(LEAD_SOURCES).map(source => (
                                <option key={source} value={source}>
                                  {source.replace('_', ' ')}
                                </option>
                              ))}
                            </select>
                            {formErrors.source && (
                              <p className="mt-1 text-sm text-red-600" id="source-error">
                                {formErrors.source}
                              </p>
                            )}
                          </div>
                        </div>

                        <div className="sm:col-span-3">
                          <label htmlFor="status" className="block text-sm font-medium text-gray-700">
                            Status
                          </label>
                          <div className="mt-1">
                            <select
                              id="status"
                              name="status"
                              required
                              value={newLead.status}
                              onChange={handleInputChange}
                              className="shadow-sm focus:ring-blue-500 focus:border-blue-500 block w-full sm:text-sm border-gray-300 rounded-md"
                            >
                              {Object.values(LEAD_STATUSES).map(status => (
                                <option key={status} value={status}>
                                  {status.replace('_', ' ')}
                                </option>
                              ))}
                            </select>
                            {formErrors.status && (
                              <p className="mt-1 text-sm text-red-600" id="status-error">
                                {formErrors.status}
                              </p>
                            )}
                          </div>
                        </div>

                        <div className="sm:col-span-6">
                          <label htmlFor="score" className="block text-sm font-medium text-gray-700">
                            Lead Score (1-5)
                          </label>
                          <div className="mt-1">
                            <input
                              type="range"
                              id="score"
                              name="score"
                              min={LEAD_SCORES.MIN}
                              max={LEAD_SCORES.MAX}
                              value={newLead.score}
                              onChange={handleInputChange}
                              className="w-full h-2 bg-gray-200 rounded-lg appearance-none cursor-pointer"
                              required
                            />
                            <div className="flex justify-between text-xs text-gray-500 px-1">
                              {Array.from({ length: LEAD_SCORES.MAX }, (_, i) => i + 1).map(num => (
                                <span key={num}>{num}</span>
                              ))}
                            </div>
                            <div className="text-center mt-2">
                              <span className={`px-2 py-1 inline-flex text-xs leading-5 font-semibold rounded-full ${getScoreColorClass(newLead.score)}`}>
                                Current Score: {newLead.score}/5
                              </span>
                            </div>
                            {formErrors.score && (
                              <p className="mt-1 text-sm text-red-600" id="score-error">
                                {formErrors.score}
                              </p>
                            )}
                          </div>
                        </div>

                        <div className="sm:col-span-6">
                          <label htmlFor="notes" className="block text-sm font-medium text-gray-700">
                            Notes
                          </label>
                          <div className="mt-1">
                            <textarea
                              id="notes"
                              name="notes"
                              rows={3}
                              value={newLead.notes}
                              onChange={handleInputChange}
                              className="shadow-sm focus:ring-blue-500 focus:border-blue-500 block w-full sm:text-sm border-gray-300 rounded-md"
                            />
                          </div>
                        </div>
                      </div>
                    </div>
                  </div>
                </div>
                <div className="bg-gray-50 px-4 py-3 sm:px-6 sm:flex sm:flex-row-reverse">
                  <button
                    type="submit"
                    disabled={loading}
                    className="inline-flex justify-center py-2 px-4 border border-transparent shadow-sm text-sm font-medium rounded-md text-white bg-blue-600 hover:bg-blue-700 focus:outline-none focus:ring-2 focus:ring-offset-2 focus:ring-blue-500"
                  >
                    {loading ? 'Adding...' : 'Add Lead'}
                  </button>
                  <button
                    type="button"
                    onClick={() => setShowAddModal(false)}
                    className="mt-3 w-full inline-flex justify-center rounded-md border border-gray-300 shadow-sm px-4 py-2 bg-white text-base font-medium text-gray-700 hover:bg-gray-50 focus:outline-none focus:ring-2 focus:ring-offset-2 focus:ring-blue-500 sm:mt-0 sm:ml-3 sm:w-auto sm:text-sm"
                  >
                    Cancel
                  </button>
                </div>
              </form>
            </div>
          </div>
        </div>
      )}
    </div>
  );
};

export default Leads;

In [ ]:
import React, { useState } from 'react';
import { Link, useNavigate } from 'react-router-dom';
import { useAuth } from '../../firebase/auth';

const Login = () => {
  const [email, setEmail] = useState('');
  const [password, setPassword] = useState('');
  const [error, setError] = useState('');
  const [loading, setLoading] = useState(false);
  const { login } = useAuth();
  const navigate = useNavigate();

  const handleSubmit = async (e) => {
    e.preventDefault();

    if (!email || !password) {
      setError('Please enter both email and password');
      return;
    }

    try {
      setError('');
      setLoading(true);
      await login(email, password);
      navigate('/dashboard');
    } catch (error) {
      console.error('Login error:', error);
      setError('Failed to log in. Please check your credentials.');
    } finally {
      setLoading(false);
    }
  };

  return (
    <div className="min-h-screen flex items-center justify-center bg-gray-50 py-12 px-4 sm:px-6 lg:px-8">
      <div className="max-w-md w-full space-y-8">
        <div>
          <h2 className="mt-6 text-center text-3xl font-extrabold text-gray-900">
            Dental AI Agent
          </h2>
          <h3 className="mt-2 text-center text-xl text-gray-600">
            Sign in to your account
          </h3>
        </div>

        {error && (
          <div className="bg-red-100 border border-red-400 text-red-700 px-4 py-3 rounded relative" role="alert">
            <span className="block sm:inline">{error}</span>
          </div>
        )}

        <form className="mt-8 space-y-6" onSubmit={handleSubmit}>
          <div className="rounded-md shadow-sm -space-y-px">
            <div>
              <label htmlFor="email-address" className="sr-only">Email address</label>
              <input
                id="email-address"
                name="email"
                type="email"
                autoComplete="email"
                required
                className="appearance-none rounded-none relative block w-full px-3 py-2 border border-gray-300 placeholder-gray-500 text-gray-900 rounded-t-md focus:outline-none focus:ring-blue-500 focus:border-blue-500 focus:z-10 sm:text-sm"
                placeholder="Email address"
                value={email}
                onChange={(e) => setEmail(e.target.value)}
              />
            </div>
            <div>
              <label htmlFor="password" className="sr-only">Password</label>
              <input
                id="password"
                name="password"
                type="password"
                autoComplete="current-password"
                required
                className="appearance-none rounded-none relative block w-full px-3 py-2 border border-gray-300 placeholder-gray-500 text-gray-900 rounded-b-md focus:outline-none focus:ring-blue-500 focus:border-blue-500 focus:z-10 sm:text-sm"
                placeholder="Password"
                value={password}
                onChange={(e) => setPassword(e.target.value)}
              />
            </div>
          </div>

          <div>
            <button
              type="submit"
              disabled={loading}
              className={`group relative w-full flex justify-center py-2 px-4 border border-transparent text-sm font-medium rounded-md text-white ${
                loading ? 'bg-blue-400' : 'bg-blue-600 hover:bg-blue-700'
              } focus:outline-none focus:ring-2 focus:ring-offset-2 focus:ring-blue-500`}
            >
              {loading ? 'Signing in...' : 'Sign in'}
            </button>
          </div>

          <div className="flex items-center justify-between">
            <div className="text-sm">
              <Link to="/register" className="font-medium text-blue-600 hover:text-blue-500">
                Don't have an account? Register
              </Link>
            </div>
          </div>
        </form>
      </div>
    </div>
  );
};

export default Login;

In [ ]:
import React, { useState, useEffect, useCallback, useMemo } from 'react';
import { useParams, useNavigate } from 'react-router-dom';
import { patientService, appointmentService, leadService, callService } from '../../firebase/firestore';
import { ToastContext } from '../../contexts/ToastContext';
import { ErrorBoundary } from '../../components/ErrorBoundary';
import { useAudio } from '../../hooks/useAudio';
import { format } from 'date-fns';

const PATIENT_FIELDS = {
  firstName: 'First Name',
  lastName: 'Last Name',
  email: 'Email',
  phone: 'Phone',
  address: 'Address',
  insuranceProvider: 'Insurance Provider',
  insuranceId: 'Insurance ID',
  medicalHistory: 'Medical History'
};

const validatePatient = (patient) => {
  const errors = {};

  if (!patient.firstName?.trim()) {
    errors.firstName = 'First name is required';
  }

  if (!patient.lastName?.trim()) {
    errors.lastName = 'Last name is required';
  }

  if (patient.email && !/^[^\s@]+@[^\s@]+\.[^\s@]+$/.test(patient.email)) {
    errors.email = 'Please enter a valid email address';
  }

  if (patient.phone && !/^[\d\s\-]+$/.test(patient.phone)) {
    errors.phone = 'Please enter a valid phone number';
  }

  if (patient.medicalHistory && patient.medicalHistory.length > 1000) {
    errors.medicalHistory = 'Medical history cannot exceed 1000 characters';
  }

  if (patient.insuranceId && !/^[A-Za-z0-9\-]+$/.test(patient.insuranceId)) {
    errors.insuranceId = 'Insurance ID can only contain letters, numbers, and dashes';
  }

  return errors;
};

const PatientDetail = () => {
  const { id } = useParams();
  const navigate = useNavigate();
  const { showToast } = React.useContext(ToastContext);
  const [patient, setPatient] = useState(null);
  const [appointments, setAppointments] = useState([]);
  const [leads, setLeads] = useState([]);
  const [calls, setCalls] = useState([]);
  const [loading, setLoading] = useState(true);
  const [error, setError] = useState(null);
  const [isEditing, setIsEditing] = useState(false);
  const [editedPatient, setEditedPatient] = useState(null);
  const [formErrors, setFormErrors] = useState({});
  const [activeTab, setActiveTab] = useState('info');
  const [tabLoading, setTabLoading] = useState({ info: false, appointments: false, leads: false, calls: false });
  const [isDeleting, setIsDeleting] = useState(false);
  const [showDeleteDialog, setShowDeleteDialog] = useState(false);

  const { playSound } = useAudio('error');

  useEffect(() => {
    const cleanup = new Set();

    const fetchPatientData = async () => {
      try {
        setLoading(true);
        setError(null);

        // Fetch patient details
        const patientData = await patientService.getPatient(id);
        if (!patientData) {
          setError('Patient not found');
          setLoading(false);
          return;
        }

        setPatient(patientData);
        setEditedPatient(patientData);

        // Fetch related data with error handling
        const [patientAppointments, patientLeads, patientCalls] = await Promise.all([
          appointmentService.getPatientAppointments(id).catch(() => []),
          leadService.getLeadsByPatient(id).catch(() => []),
          callService.getPatientCalls(id).catch(() => [])
        ]);

        setAppointments(patientAppointments);
        setLeads(patientLeads);
        setCalls(patientCalls);

        setLoading(false);
      } catch (error) {
        console.error('Error fetching patient data:', error);
        setError('Failed to load patient data. Please try again.');
        setLoading(false);
      }
    };

    fetchPatientData();

    return () => {
      cleanup.forEach(fn => fn());
    };
  }, [id]);

  const handleInputChange = useCallback((e) => {
    const { name, value } = e.target;
    setEditedPatient(prev => ({
      ...prev,
      [name]: value.trim()
    }));
    setFormErrors(prev => ({
      ...prev,
      [name]: ''
    }));
  }, []);

  const handleSave = useCallback(async () => {
    const validationErrors = validatePatient(editedPatient);
    if (Object.keys(validationErrors).length > 0) {
      setFormErrors(validationErrors);
      playSound();
      return;
    }

    try {
      setTabLoading(prev => ({ ...prev, info: true }));

      // Update patient in Firestore
      await patientService.updatePatient(id, editedPatient);

      // Update local state
      setPatient(editedPatient);
      setIsEditing(false);
      setFormErrors({});

      showToast({
        message: 'Patient information updated successfully',
        type: 'success'
      });
    } catch (error) {
      console.error('Error updating patient:', error);
      setError('Failed to update patient. Please try again.');
      playSound();
    } finally {
      setTabLoading(prev => ({ ...prev, info: false }));
    }
  }, [editedPatient, id, playSound, showToast]);

  const handleDelete = useCallback(async () => {
    if (window.confirm('Are you sure you want to delete this patient? This action cannot be undone.')) {
      try {
        setIsDeleting(true);

        // Delete patient from Firestore
        await patientService.deletePatient(id);

        // Navigate back to patients list
        showToast({
          message: 'Patient deleted successfully',
          type: 'success'
        });
        navigate('/patients');
      } catch (error) {
        console.error('Error deleting patient:', error);
        setError('Failed to delete patient. Please try again.');
        playSound();
      } finally {
        setIsDeleting(false);
      }
    }
  }, [id, navigate, playSound, showToast]);

  const renderInfoTab = useMemo(() => {
    if (!patient) return null;

    return (
      <div className="bg-white shadow overflow-hidden sm:rounded-lg">
        <div className="px-4 py-5 sm:px-6">
          <h3 className="text-lg leading-6 font-medium text-gray-900">Patient Information</h3>
          <p className="mt-1 max-w-2xl text-sm text-gray-500">Personal details and medical history.</p>
        </div>
        <div className="border-t border-gray-200">
          {isEditing ? (
            <div className="px-4 py-5 sm:p-6">
              <div className="grid grid-cols-1 gap-y-6 gap-x-4 sm:grid-cols-6">
                {Object.entries(PATIENT_FIELDS).map(([name, label]) => (
                  <div key={name} className={`sm:col-span-${name === 'medicalHistory' ? 6 : 3}`}>
                    <label htmlFor={name} className="block text-sm font-medium text-gray-700">
                      {label}
                    </label>
                    <div className="mt-1">
                      {name === 'medicalHistory' ? (
                        <textarea
                          id={name}
                          name={name}
                          rows={4}
                          value={editedPatient[name] || ''}
                          onChange={handleInputChange}
                          className={`shadow-sm focus:ring-blue-500 focus:border-blue-500 block w-full sm:text-sm border-gray-300 rounded-md
                            ${formErrors[name] && 'border-red-500'}`}
                          aria-invalid={formErrors[name] ? 'true' : 'false'}
                          aria-describedby={`${name}-error`}
                        />
                      ) : (
                        <input
                          type={name === 'email' ? 'email' : name === 'phone' ? 'tel' : 'text'}
                          name={name}
                          id={name}
                          value={editedPatient[name] || ''}
                          onChange={handleInputChange}
                          className={`shadow-sm focus:ring-blue-500 focus:border-blue-500 block w-full sm:text-sm border-gray-300 rounded-md
                            ${formErrors[name] && 'border-red-500'}`}
                          aria-invalid={formErrors[name] ? 'true' : 'false'}
                          aria-describedby={`${name}-error`}
                        />
                      )}
                      {formErrors[name] && (
                        <p id={`${name}-error`} className="mt-1 text-sm text-red-600">
                          {formErrors[name]}
                        </p>
                      )}
                    </div>
                  </div>
                ))}
              </div>
            </div>
          ) : (
            <dl>
              <div className="bg-gray-50 px-4 py-5 sm:grid sm:grid-cols-3 sm:gap-4 sm:px-6">
                <dt className="text-sm font-medium text-gray-500">Full name</dt>
                <dd className="mt-1 text-sm text-gray-900 sm:mt-0 sm:col-span-2">
                  {patient.firstName} {patient.lastName}
                </dd>
              </div>
              <div className="bg-white px-4 py-5 sm:grid sm:grid-cols-3 sm:gap-4 sm:px-6">
                <dt className="text-sm font-medium text-gray-500">Email address</dt>
                <dd className="mt-1 text-sm text-gray-900 sm:mt-0 sm:col-span-2">
                  {patient.email || 'Not provided'}
                </dd>
              </div>
              <div className="bg-gray-50 px-4 py-5 sm:grid sm:grid-cols-3 sm:gap-4 sm:px-6">
                <dt className="text-sm font-medium text-gray-500">Phone number</dt>
                <dd className="mt-1 text-sm text-gray-900 sm:mt-0 sm:col-span-2">
                  {patient.phone || 'Not provided'}
                </dd>
              </div>
              <div className="bg-white px-4 py-5 sm:grid sm:grid-cols-3 sm:gap-4 sm:px-6">
                <dt className="text-sm font-medium text-gray-500">Address</dt>
                <dd className="mt-1 text-sm text-gray-900 sm:mt-0 sm:col-span-2">
                  {patient.address || 'Not provided'}
                </dd>
              </div>
              <div className="bg-gray-50 px-4 py-5 sm:grid sm:grid-cols-3 sm:gap-4 sm:px-6">
                <dt className="text-sm font-medium text-gray-500">Insurance Provider</dt>
                <dd className="mt-1 text-sm text-gray-900 sm:mt-0 sm:col-span-2">
                  {patient.insuranceProvider || 'Not provided'}
                </dd>
              </div>
              <div className="bg-white px-4 py-5 sm:grid sm:grid-cols-3 sm:gap-4 sm:px-6">
                <dt className="text-sm font-medium text-gray-500">Insurance ID</dt>
                <dd className="mt-1 text-sm text-gray-900 sm:mt-0 sm:col-span-2">
                  {patient.insuranceId || 'Not provided'}
                </dd>
              </div>
              <div className="bg-gray-50 px-4 py-5 sm:grid sm:grid-cols-3 sm:gap-4 sm:px-6">
                <dt className="text-sm font-medium text-gray-500">Medical History</dt>
                <dd className="mt-1 text-sm text-gray-900 sm:mt-0 sm:col-span-2">
                  {patient.medicalHistory || 'Not provided'}
                </dd>
              </div>
            </dl>
          )}
        </div>
      </div>
    );
  }, [patient, isEditing, editedPatient, handleInputChange, formErrors]);

  if (loading && !patient) {
    return (
      <div className="flex items-center justify-center h-64">
        <div className="animate-spin rounded-full h-12 w-12 border-t-2 border-b-2 border-blue-500"></div>
      </div>
    );
  }

  if (error) {
    return (
      <div className="bg-red-100 border border-red-400 text-red-700 px-4 py-3 rounded relative" role="alert">
        <span className="block sm:inline">{error}</span>
      </div>
    );
  }

  if (!patient) {
    return (
      <div className="bg-yellow-100 border border-yellow-400 text-yellow-700 px-4 py-3 rounded relative" role="alert">
        <span className="block sm:inline">Patient not found</span>
      </div>
    );
  }

  return (
    <ErrorBoundary>
      <div>
        <div className="flex justify-between items-center">
          <h1 className="text-2xl font-semibold text-gray-900">
            {isEditing ? 'Edit Patient' : `${patient.firstName} ${patient.lastName}`}
          </h1>
          <div className="flex space-x-2">
            {isEditing ? (
              <>
                <button
                  onClick={handleSave}
                  disabled={tabLoading.info || isDeleting}
                  className="inline-flex items-center px-4 py-2 border border-transparent text-sm font-medium rounded-md shadow-sm text-white bg-blue-600 hover:bg-blue-700 focus:outline-none focus:ring-2 focus:ring-offset-2 focus:ring-blue-500"
                >
                  {tabLoading.info ? 'Saving...' : 'Save'}
                </button>
                <button
                  onClick={() => {
                    setEditedPatient(patient);
                    setIsEditing(false);
                    setFormErrors({});
                  }}
                  className="inline-flex items-center px-4 py-2 border border-gray-300 text-sm font-medium rounded-md shadow-sm text-gray-700 bg-white hover:bg-gray-50 focus:outline-none focus:ring-2 focus:ring-offset-2 focus:ring-blue-500"
                >
                  Cancel
                </button>
              </>
            ) : (
              <>
                <button
                  onClick={() => setIsEditing(true)}
                  className="inline-flex items-center px-4 py-2 border border-transparent text-sm font-medium rounded-md shadow-sm text-white bg-blue-600 hover:bg-blue-700 focus:outline-none focus:ring-2 focus:ring-offset-2 focus:ring-blue-500"
                >
                  Edit
                </button>
                <button
                  onClick={() => setShowDeleteDialog(true)}
                  disabled={isDeleting}
                  className="inline-flex items-center px-4 py-2 border border-transparent text-sm font-medium rounded-md shadow-sm text-white bg-red-600 hover:bg-red-700 focus:outline-none focus:ring-2 focus:ring-offset-2 focus:ring-red-500"
                >
                  {isDeleting ? 'Deleting...' : 'Delete'}
                </button>
              </>
            )}
          </div>
        </div>

        {/* Tabs */}
        <div className="mt-6 border-b border-gray-200">
          <nav className="-mb-px flex space-x-8">
            {['info', 'appointments', 'leads', 'calls'].map((tab) => (
              <button
                key={tab}
                onClick={() => setActiveTab(tab)}
                className={`whitespace-nowrap py-4 px-1 border-b-2 font-medium text-sm
                  ${activeTab === tab
                    ? 'border-blue-500 text-blue-600'
                    : 'border-transparent text-gray-500 hover:text-gray-700 hover:border-gray-300'
                  }`}
              >
                {tab.charAt(0).toUpperCase() + tab.slice(1)}
                {tab !== 'info' && (
                  <span className="ml-2 text-gray-500">
                    {tab === 'appointments' ? appointments.length :
                     tab === 'leads' ? leads.length :
                     tab === 'calls' ? calls.length : 0}
                  </span>
                )}
              </button>
            ))}
          </nav>
        </div>

        {/* Tab Content */}
        <div className="mt-6">
          {renderInfoTab}
          {/* Add other tab content components here */}
        </div>

        {/* Delete Confirmation Dialog */}
        <div
          className={`fixed inset-0 bg-black bg-opacity-50 z-50 ${showDeleteDialog ? 'block' : 'hidden'}`}
          onClick={() => setShowDeleteDialog(false)}
          aria-hidden="true"
        />
        <div
          className={`fixed inset-0 flex items-center justify-center z-50 ${showDeleteDialog ? 'block' : 'hidden'}`}
        >
          <div className="bg-white rounded-lg p-6 max-w-md w-full mx-4">
            <div className="text-center">
              <h3 className="text-lg leading-6 font-medium text-gray-900">Delete Patient</h3>
              <div className="mt-2">
                <p className="text-sm text-gray-500">
                  Are you sure you want to delete {patient.firstName} {patient.lastName}? This action cannot be undone.
                </p>
              </div>
              <div className="mt-4 flex justify-center space-x-4">
                <button
                  onClick={() => setShowDeleteDialog(false)}
                  className="inline-flex items-center px-4 py-2 border border-gray-300 text-sm font-medium rounded-md text-gray-700 bg-white hover:bg-gray-50 focus:outline-none focus:ring-2 focus:ring-offset-2 focus:ring-blue-500"
                >
                  Cancel
                </button>
                <button
                  onClick={handleDelete}
                  className="inline-flex items-center px-4 py-2 border border-transparent text-sm font-medium rounded-md text-white bg-red-600 hover:bg-red-700 focus:outline-none focus:ring-2 focus:ring-offset-2 focus:ring-red-500"
                >
                  {isDeleting ? 'Deleting...' : 'Delete'}
                </button>
              </div>
            </div>
          </div>
        </div>
      </div>
    </ErrorBoundary>
  );
};

export default PatientDetail;

In [ ]:
import React, { useState, useEffect, useCallback, useMemo } from 'react';
import { Link } from 'react-router-dom';
import { patientService } from '../../firebase/firestore';
import { useApi } from '../../services/api';
import { ToastContext } from '../../contexts/ToastContext';
import { ErrorBoundary } from '../../components/ErrorBoundary';
import { useAudio } from '../../hooks/useAudio';
import { validateEmail, validatePhone } from '../../utils/validators';
import { format } from 'date-fns';

// Constants
const PATIENT_FIELDS = {
  firstName: 'First Name',
  lastName: 'Last Name',
  email: 'Email',
  phone: 'Phone',
  address: 'Address',
  insuranceProvider: 'Insurance Provider',
  insuranceId: 'Insurance ID',
  medicalHistory: 'Medical History'
};

const validatePatient = (patient) => {
  const errors = {};

  if (!patient.firstName?.trim()) {
    errors.firstName = 'First name is required';
  }

  if (!patient.lastName?.trim()) {
    errors.lastName = 'Last name is required';
  }

  if (patient.email && !validateEmail(patient.email)) {
    errors.email = 'Please enter a valid email address';
  }

  if (patient.phone && !validatePhone(patient.phone)) {
    errors.phone = 'Please enter a valid phone number';
  }

  if (patient.medicalHistory && patient.medicalHistory.length > 1000) {
    errors.medicalHistory = 'Medical history cannot exceed 1000 characters';
  }

  if (patient.insuranceId && !/^[A-Za-z0-9\-]+$/.test(patient.insuranceId)) {
    errors.insuranceId = 'Insurance ID can only contain letters, numbers, and dashes';
  }

  return errors;
};

const Patients = () => {
  const [patients, setPatients] = useState([]);
  const [loading, setLoading] = useState(true);
  const [error, setError] = useState(null);
  const [searchTerm, setSearchTerm] = useState('');
  const [showAddModal, setShowAddModal] = useState(false);
  const [newPatient, setNewPatient] = useState({
    firstName: '',
    lastName: '',
    email: '',
    phone: '',
    address: '',
    insuranceProvider: '',
    insuranceId: '',
    medicalHistory: ''
  });
  const [formErrors, setFormErrors] = useState({});
  const [isSubmitting, setIsSubmitting] = useState(false);
  const [searchLoading, setSearchLoading] = useState(false);
  const [page, setPage] = useState(1);
  const [totalPages, setTotalPages] = useState(1);

  const { showToast } = React.useContext(ToastContext);
  const { playSound } = useAudio('error');
  const api = useApi();

  useEffect(() => {
    let isMounted = true;

    const fetchPatients = async () => {
      try {
        setLoading(true);
        setError(null);

        // Fetch from Firestore with pagination
        const { data, total } = await patientService.getAllPatients({
          limit: 10,
          page: page
        });

        if (isMounted) {
          setPatients(data);
          setTotalPages(Math.ceil(total / 10));
        }

      } catch (error) {
        console.error('Error fetching patients:', error);
        setError('Failed to load patients. Please try again.');
        playSound();
      } finally {
        if (isMounted) setLoading(false);
      }
    };

    fetchPatients();

    return () => {
      isMounted = false;
    };
  }, [page]);

  const handleSearchChange = useCallback((e) => {
    const value = e.target.value;
    setSearchTerm(value);
    setPage(1);
    setSearchLoading(true);

    // Debounce search
    clearTimeout(window.searchTimeout);
    window.searchTimeout = setTimeout(() => {
      setSearchLoading(false);
    }, 300);
  }, []);

  const filteredPatients = useMemo(() => {
    if (!searchTerm) return patients;

    return patients.filter(patient => {
      const fullName = `${patient.firstName} ${patient.lastName}`.toLowerCase();
      return fullName.includes(searchTerm.toLowerCase()) ||
             (patient.email && patient.email.toLowerCase().includes(searchTerm.toLowerCase())) ||
             (patient.phone && patient.phone.toLowerCase().includes(searchTerm.toLowerCase()));
    });
  }, [patients, searchTerm]);

  const handleInputChange = useCallback((e) => {
    const { name, value } = e.target;
    setNewPatient(prev => ({
      ...prev,
      [name]: value.trim()
    }));
    setFormErrors(prev => ({
      ...prev,
      [name]: ''
    }));
  }, []);

  const handleSubmit = useCallback(async (e) => {
    e.preventDefault();

    const validationErrors = validatePatient(newPatient);
    if (Object.keys(validationErrors).length > 0) {
      setFormErrors(validationErrors);
      playSound();
      return;
    }

    try {
      setIsSubmitting(true);
      setError(null);

      // Add to Firestore
      const patientId = await patientService.addPatient(newPatient);

      // Add the new patient to the state
      setPatients(prev => [
        ...prev,
        { id: patientId, ...newPatient, createdAt: new Date() }
      ]);

      // Reset form and close modal
      setNewPatient({
        firstName: '',
        lastName: '',
        email: '',
        phone: '',
        address: '',
        insuranceProvider: '',
        insuranceId: '',
        medicalHistory: ''
      });
      setShowAddModal(false);
      showToast({
        message: 'Patient added successfully',
        type: 'success'
      });

    } catch (error) {
      console.error('Error adding patient:', error);
      setError('Failed to add patient. Please try again.');
      playSound();
    } finally {
      setIsSubmitting(false);
    }
  }, [newPatient, playSound, showToast]);

  const handlePageChange = useCallback((newPage) => {
    setPage(newPage);
  }, []);

  if (loading && patients.length === 0) {
    return (
      <div className="flex items-center justify-center h-64">
        <div className="animate-spin rounded-full h-12 w-12 border-t-2 border-b-2 border-blue-500"></div>
      </div>
    );
  }

  return (
    <ErrorBoundary>
      <div>
        <div className="flex justify-between items-center">
          <h1 className="text-2xl font-semibold text-gray-900">Patients</h1>
          <button
            onClick={() => setShowAddModal(true)}
            disabled={isSubmitting}
            className="inline-flex items-center px-4 py-2 border border-transparent text-sm font-medium rounded-md shadow-sm text-white bg-blue-600 hover:bg-blue-700 focus:outline-none focus:ring-2 focus:ring-offset-2 focus:ring-blue-500 disabled:opacity-50"
          >
            <svg className="-ml-1 mr-2 h-5 w-5" fill="none" viewBox="0 0 24 24" stroke="currentColor">
              <path strokeLinecap="round" strokeLinejoin="round" strokeWidth="2" d="M12 6v6m0 0v6m0-6h6m-6 0H6" />
            </svg>
            Add Patient
          </button>
        </div>

        {/* Search Bar */}
        <div className="mt-4">
          <div className="relative rounded-md shadow-sm">
            <div className="absolute inset-y-0 left-0 pl-3 flex items-center pointer-events-none">
              <svg className="h-5 w-5 text-gray-400" fill="none" viewBox="0 0 24 24" stroke="currentColor">
                <path strokeLinecap="round" strokeLinejoin="round" strokeWidth="2" d="M21 21l-6-6m2-5a7 7 0 11-14 0 7 7 0 0114 0z" />
              </svg>
            </div>
            <input
              type="text"
              className="focus:ring-blue-500 focus:border-blue-500 block w-full pl-10 sm:text-sm border-gray-300 rounded-md"
              placeholder="Search patients..."
              value={searchTerm}
              onChange={handleSearchChange}
              aria-label="Search patients"
              disabled={searchLoading}
            />
          </div>
        </div>

        {error && (
          <div className="mt-4 bg-red-100 border border-red-400 text-red-700 px-4 py-3 rounded relative" role="alert">
            <span className="block sm:inline">{error}</span>
          </div>
        )}

        {/* Patients List */}
        <div className="mt-6 bg-white shadow overflow-hidden sm:rounded-md">
          <ul className="divide-y divide-gray-200">
            {filteredPatients.length > 0 ? (
              filteredPatients.map((patient) => (
                <li key={patient.id}>
                  <Link
                    to={`/patients/${patient.id}`}
                    className="block hover:bg-gray-50"
                    aria-label={`View ${patient.firstName} ${patient.lastName} details`}
                  >
                    <div className="px-4 py-4 sm:px-6">
                      <div className="flex items-center justify-between">
                        <div className="text-sm font-medium text-blue-600 truncate">
                          {patient.firstName} {patient.lastName}
                        </div>
                      </div>
                      <div className="mt-2 sm:flex sm:justify-between">
                        <div className="sm:flex">
                          {patient.email && (
                            <div className="flex items-center text-sm text-gray-500 mr-6">
                              <svg className="flex-shrink-0 mr-1.5 h-5 w-5 text-gray-400" fill="none" viewBox="0 0 24 24" stroke="currentColor">
                                <path strokeLinecap="round" strokeLinejoin="round" strokeWidth="2" d="M3 8l7.89 5.26a2 2 0 002.22 0L21 8M5 19h14a2 2 0 002-2V7a2 2 0 00-2-2H5a2 2 0 00-2 2v10a2 2 0 002 2z" />
                              </svg>
                              {patient.email}
                            </div>
                          )}
                          {patient.phone && (
                            <div className="mt-2 flex items-center text-sm text-gray-500 sm:mt-0">
                              <svg className="flex-shrink-0 mr-1.5 h-5 w-5 text-gray-400" fill="none" viewBox="0 0 24 24" stroke="currentColor">
                                <path strokeLinecap="round" strokeLinejoin="round" strokeWidth="2" d="M3 5a2 2 0 012-2h3.28a1 1 0 01.948.684l1.498 4.493a1 1 0 01-.502 1.21l-2.257 1.13a11.042 11.042 0 005.516 5.516l1.13-2.257a1 1 0 011.21-.502l4.493 1.498a1 1 0 01.684.949V19a2 2 0 01-2 2h-1C9.716 21 3 14.284 3 6V5z" />
                              </svg>
                              {patient.phone}
                            </div>
                          )}
                        </div>
                        {patient.insuranceProvider && (
                          <div className="mt-2 flex items-center text-sm text-gray-500 sm:mt-0">
                            <svg className="flex-shrink-0 mr-1.5 h-5 w-5 text-gray-400" fill="none" viewBox="0 0 24 24" stroke="currentColor">
                              <path strokeLinecap="round" strokeLinejoin="round" strokeWidth="2" d="M9 12l2 2 4-4m5.618-4.016A11.955 11.955 0 0112 2.944a11.955 11.955 0 01-8.618 3.04A12.02 12.02 0 003 9c0 5.591 3.824 10.29 9 11.622 5.176-1.332 9-6.03 9-11.622 0-1.042-.133-2.052-.382-3.016z" />
                            </svg>
                            {patient.insuranceProvider}
                          </div>
                        )}
                      </div>
                    </div>
                  </Link>
                </li>
              ))
            ) : (
              <li className="px-4 py-4 sm:px-6 text-sm text-gray-500">
                {searchTerm ? 'No patients match your search.' : 'No patients found.'}
              </li>
            )}
          </ul>
        </div>

        {/* Pagination */}
        {totalPages > 1 && (
          <div className="mt-4 flex justify-center">
            <nav className="relative z-0 inline-flex rounded-md shadow-sm -space-x-px" aria-label="Pagination">
              <button
                onClick={() => handlePageChange(page - 1)}
                disabled={page === 1}
                className="relative inline-flex items-center px-2 py-2 rounded-l-md border border-gray-300 bg-white text-sm font-medium text-gray-500 hover:bg-gray-50 disabled:opacity-50"
              >
                Previous
              </button>
              {Array.from({ length: totalPages }, (_, i) => i + 1).map((num) => (
                <button
                  key={num}
                  onClick={() => handlePageChange(num)}
                  className={`relative inline-flex items-center px-4 py-2 border text-sm font-medium ${
                    page === num
                      ? 'z-10 bg-blue-50 border-blue-500 text-blue-600'
                      : 'bg-white border-gray-300 text-gray-500 hover:bg-gray-50'
                  }`}
                >
                  {num}
                </button>
              ))}
              <button
                onClick={() => handlePageChange(page + 1)}
                disabled={page === totalPages}
                className="relative inline-flex items-center px-2 py-2 rounded-r-md border border-gray-300 bg-white text-sm font-medium text-gray-500 hover:bg-gray-50 disabled:opacity-50"
              >
                Next
              </button>
            </nav>
          </div>
        )}

        {/* Add Patient Modal */}
        {showAddModal && (
          <div className="fixed z-10 inset-0 overflow-y-auto">
            <div className="flex items-end justify-center min-h-screen pt-4 px-4 pb-20 text-center sm:block sm:p-0">
              <div className="fixed inset-0 transition-opacity" aria-hidden="true">
                <div className="absolute inset-0 bg-gray-500 opacity-75"></div>
              </div>

              <span className="hidden sm:inline-block sm:align-middle sm:h-screen" aria-hidden="true">&#8203;</span>

              <div className="inline-block align-bottom bg-white rounded-lg text-left overflow-hidden shadow-xl transform transition-all sm:my-8 sm:align-middle sm:max-w-lg sm:w-full">
                <form onSubmit={handleSubmit}>
                  <div className="bg-white px-4 pt-5 pb-4 sm:p-6 sm:pb-4">
                    <div className="sm:flex sm:items-start">
                      <div className="mt-3 text-center sm:mt-0 sm:ml-4 sm:text-left w-full">
                        <h3 className="text-lg leading-6 font-medium text-gray-900" id="modal-title">
                          Add New Patient
                        </h3>
                        <div className="mt-4 grid grid-cols-1 gap-y-6 gap-x-4 sm:grid-cols-6">
                          {Object.entries(PATIENT_FIELDS).map(([name, label]) => (
                            <div key={name} className={`sm:col-span-${name === 'medicalHistory' ? 6 : 3}`}>
                              <label htmlFor={name} className="block text-sm font-medium text-gray-700">
                                {label}
                              </label>
                              <div className="mt-1">
                                {name === 'medicalHistory' ? (
                                  <textarea
                                    id={name}
                                    name={name}
                                    rows={4}
                                    value={newPatient[name]}
                                    onChange={handleInputChange}
                                    className={`shadow-sm focus:ring-blue-500 focus:border-blue-500 block w-full sm:text-sm border-gray-300 rounded-md
                                      ${formErrors[name] && 'border-red-500'}`}
                                    aria-invalid={formErrors[name] ? 'true' : 'false'}
                                    aria-describedby={`${name}-error`}
                                  />
                                ) : (
                                  <input
                                    type={name === 'email' ? 'email' : name === 'phone' ? 'tel' : 'text'}
                                    name={name}
                                    id={name}
                                    value={newPatient[name]}
                                    onChange={handleInputChange}
                                    className={`shadow-sm focus:ring-blue-500 focus:border-blue-500 block w-full sm:text-sm border-gray-300 rounded-md
                                      ${formErrors[name] && 'border-red-500'}`}
                                    aria-invalid={formErrors[name] ? 'true' : 'false'}
                                    aria-describedby={`${name}-error`}
                                  />
                                )}
                                {formErrors[name] && (
                                  <p id={`${name}-error`} className="mt-1 text-sm text-red-600">
                                    {formErrors[name]}
                                  </p>
                                )}
                              </div>
                            </div>
                          ))}
                        </div>
                      </div>
                    </div>
                  </div>
                  <div className="bg-gray-50 px-4 py-3 sm:px-6 sm:flex sm:flex-row-reverse">
                    <button
                      type="submit"
                      disabled={isSubmitting || Object.keys(formErrors).length > 0}
                      className={`inline-flex justify-center w-full rounded-md border border-transparent px-4 py-2 bg-blue-600 text-base font-medium text-white shadow-sm hover:bg-blue-700 focus:outline-none focus:ring-2 focus:ring-offset-2 focus:ring-blue-500 sm:w-auto sm:text-sm ${
                        isSubmitting || Object.keys(formErrors).length > 0 ? 'opacity-50 cursor-not-allowed' : ''
                      }`}
                    >
                      {isSubmitting ? 'Adding...' : 'Add Patient'}
                    </button>
                    <button
                      type="button"
                      onClick={() => {
                        setNewPatient({
                          firstName: '',
                          lastName: '',
                          email: '',
                          phone: '',
                          address: '',
                          insuranceProvider: '',
                          insuranceId: '',
                          medicalHistory: ''
                        });
                        setFormErrors({});
                        setShowAddModal(false);
                      }}
                      className="mt-3 w-full inline-flex justify-center rounded-md border border-gray-300 px-4 py-2 bg-white text-base font-medium text-gray-700 shadow-sm hover:bg-gray-50 focus:outline-none focus:ring-2 focus:ring-offset-2 focus:ring-blue-500 sm:mt-0 sm:ml-3 sm:w-auto sm:text-sm"
                    >
                      Cancel
                    </button>
                  </div>
                </form>
              </div>
            </div>
          </div>
        )}
      </div>
    </ErrorBoundary>
  );
};

export default Patients;